In [3]:
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np
import os
from xlwt import Workbook #xử lý với file excel .xls
import xlwt
import talib
import time
import datetime
from datetime import date
from datetime import timedelta
today = date.today()
str_today = date.today().strftime("%d-%m-%Y")
str_today_weekday =  date.today().strftime("%d-%m-%Y-%A")
#Tạo folder chứa file hôm nay
#os.mkdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')


tickers_list = ['ATVI', 'ADBE', 'AMD', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 
                'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CERN', 'CHTR', 'CHKP', 
                'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CPRT', 'COST', 'CRWD', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 
                'FB', 'FAST', 'FISV', 'FOXA', 'FOX', 'GILD', 'HON', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP',
                'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MELI', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MNST', 
                'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX', 'PYPL', 'PTON', 'PEP', 'PDD', 'QCOM', 'REGN', 
                'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 
                'WBA', 'WDAY', 'XEL', 'XLNX', 'ZM'] 

data = pd.DataFrame(columns=tickers_list) #Create placeholder for data

tickers_false = [] #Tạo list chứa những tickers ko get được dữ liệu từ yahoo finance

print(f'Đang thống kê {len(tickers_list)} cổ phiếu (Lấy dữ liệu: Close, High, Low, Volume, Open)')
STT=1
# Fetch the data
for ticker in tickers_list:
    print(f'------------- {ticker} ------------- Loading ------------- {STT}/{len(tickers_list)} -------------')
    STT+=1
    
    try:
        stock = yf.Ticker(ticker)
        data_year = stock.history(period='1y')
        if data_year.empty:
            raise Exception
    except:
        print(ticker, ' không có dữ liệu')
        tickers_false.append(ticker)
        continue
    '''
    try:
        data = yf.Ticker("qwedqwe").history(period="3mo")

        if 'Empty DataFrame' in str(data):
            raise Exception('không có dữ liệu')
    except Exception as e:
        print(e)
    data        
    '''



    #Tạo columns giá Close, High, Low
    data[ticker] = data_year['Close']
    data['High ' + ticker] = data_year['High']
    data['Low ' + ticker] = data_year['Low']
    data['Volume ' + ticker] = data_year['Volume']
    data['Open ' + ticker] = data_year['Open']
    
    #Calculate the CCI
    typical_price = (data[ticker] + data['High ' + ticker] + data['Low ' + ticker]) / 3
    data['TP ' + ticker] = typical_price
    CCI = (typical_price - typical_price.rolling(20).mean()) / (0.015*typical_price.rolling(20).std())
    data['CCI ' + ticker] = CCI
    #print('Hoàn thành tính CCI')
    
    #Calculate the RSI
    delta = data[ticker].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    time_period = 14
    AVG_Gain = up.ewm(com=time_period).mean()
    AVG_Loss = abs(down.ewm(com=time_period).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data['RSI ' + ticker]=RSI
    #print('Hoàn thành tính RSI')
    
    #Calculate the MACD
    EMA12=data[ticker].ewm(span=12, adjust=False).mean()
    data['EMA12 ' + ticker]=EMA12
    EMA26=data[ticker].ewm(span=26, adjust=False).mean()
    data['EMA26 ' + ticker]=EMA26
    MACD = EMA12 - EMA26
    data['MACD ' + ticker] = MACD
    signal = MACD.ewm(span=9, adjust=False).mean()
    data['Signal ' + ticker] = signal
    #print('Hoàn thành tính MACD')
    
    #Calculate the EMA36
    EMA36=data[ticker].ewm(span=36, adjust=False).mean()
    data['EMA36 ' + ticker]=EMA36
    #print('Hoàn thành tính EMA12 & EMA36')
    
    #Calculate the BB
    data['SMA20 ' + ticker] = data[ticker].rolling(window=20).mean()
    data['STD ' + ticker] = data[ticker].rolling(window=20).std()
    data['Upper ' + ticker] = data['SMA20 ' + ticker] + (data['STD ' + ticker]*2)
    data['Lower ' + ticker] = data['SMA20 ' + ticker] - (data['STD ' + ticker]*2)
    #print('Hoàn thành tính BB')

    #Calculate the Stochastic
    high14 = data['High ' + ticker].rolling(14).max()
    low14 = data['Low ' + ticker].rolling(14).min()
    data['%K ' + ticker] = (data[ticker] - low14) * 100/(high14 - low14)
    data['%D ' + ticker] = data['%K ' + ticker].rolling(3).mean()
    #print('Hoàn thành tính Stochastic')
        
    #Calculate the Parabolic SAR
    data['SAR ' + ticker] = talib.SAR(data['High ' + ticker], data['Low ' + ticker], acceleration=0.02, maximum=0.2)
    #print('Hoàn thành tính SAR')

    #Calculate the ADX
    data['ADX ' + ticker] = talib.ADX(data['High ' + ticker],data['Low ' + ticker], data['Open ' + ticker], timeperiod=14)
    #print('Hoàn thành tính ADX')
    
    #Calculate the SMA50
    data['SMA50 ' + ticker] = data[ticker].rolling(window=50).mean()
    #print('Hoàn thành tính SMA20 & SMA50')

#làm sạch tickers_list: xoá những tickers ko có dữ liệu
for i in tickers_false:
    tickers_list.remove(i)    

#Bỏ 120 ngày đầu
data = data[120:]
data

Đang thống kê 102 cổ phiếu (Lấy dữ liệu: Close, High, Low, Volume, Open)
------------- ATVI ------------- Loading ------------- 1/102 -------------
------------- ADBE ------------- Loading ------------- 2/102 -------------
------------- AMD ------------- Loading ------------- 3/102 -------------
------------- ALGN ------------- Loading ------------- 4/102 -------------
------------- GOOGL ------------- Loading ------------- 5/102 -------------
------------- GOOG ------------- Loading ------------- 6/102 -------------
------------- AMZN ------------- Loading ------------- 7/102 -------------
------------- AEP ------------- Loading ------------- 8/102 -------------
------------- AMGN ------------- Loading ------------- 9/102 -------------
------------- ADI ------------- Loading ------------- 10/102 -------------
------------- ANSS ------------- Loading ------------- 11/102 -------------
------------- AAPL ------------- Loading ------------- 12/102 -------------
------------- AMAT -------

,ATVI,ADBE,AMD,ALGN,GOOGL,GOOG,AMZN,AEP,AMGN,ADI,...,EMA36 ZM,SMA20 ZM,STD ZM,Upper ZM,Lower ZM,%K ZM,%D ZM,SAR ZM,ADX ZM,SMA50 ZM
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-12,95.676743,506.029999,78.580002,601.109985,2244.620117,2254.790039,3379.389893,84.571449,245.307587,158.388275,...,341.423004,325.185002,10.035479,345.255961,305.114043,42.568785,50.635907,306.497034,15.079477,361.752802
2021-04-13,97.070000,514.859985,80.190002,595.630005,2254.429932,2267.270020,3400.000000,85.279495,246.302841,157.823059,...,341.494733,324.822502,9.187013,343.196528,306.448476,97.685301,61.783891,307.145294,14.324560,361.166401
2021-04-14,96.690002,510.630005,78.550003,594.929993,2241.909912,2254.840088,3333.000000,85.623672,245.977646,157.128952,...,340.830154,324.528503,8.932027,342.392557,306.664450,62.950858,67.734981,308.607802,13.704157,360.111802
2021-04-15,97.779999,523.250000,83.010002,606.229980,2285.250000,2296.659912,3379.090088,86.351379,251.407257,159.360031,...,340.373389,324.315004,8.674357,341.663717,306.966291,70.398151,77.011437,310.792533,13.043403,358.922602
2021-04-16,96.480003,525.080017,82.150002,606.330017,2282.750000,2297.760010,3399.439941,87.079094,251.978806,158.170105,...,339.809963,324.996504,8.547666,342.091836,307.901172,64.707299,66.018769,312.846181,12.384660,357.912202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11,76.320000,573.070007,104.680000,631.849976,2778.280029,2776.949951,3246.300049,82.510002,206.699997,167.899994,...,287.360632,268.435000,11.135159,290.705319,246.164681,12.760751,15.440303,271.376229,31.012819,310.901397
2021-10-12,74.919998,580.690002,105.040001,627.429993,2728.979980,2734.260010,3247.330078,82.550003,201.889999,164.020004,...,285.637895,267.137500,11.051593,289.240686,245.034314,17.785723,15.483725,270.525580,31.366066,308.431998
2021-10-13,75.250000,591.119995,109.160004,596.219971,2751.639893,2758.000000,3284.280029,83.459999,203.610001,164.350006,...,284.589360,266.493500,10.685817,287.865134,245.121866,62.543598,31.030024,269.708956,30.036680,306.259197


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
# CCI bắt đáy: cắt lên -100, bắt đỉnh cắt xuống 100
### Thông báo: Biến động giá mạnh trong thời gian ngắn
### Khoảng 2,3 ngày sau nếu MACD cắt lên Signal thì tín hiệu tốt
### Lọc trong vòng 1 tuần
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


In [4]:
first_cross = 0
df_CCI = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
    
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data['CCI ' + ticker][i] < 100 and data['CCI ' + ticker][i] > 0 and first_cross == 0:
            print(ticker, '=> CCI cắt xuống 100 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1 
            df_CCI = df_CCI.append(dict(zip(df_CCI.columns,[ticker, 'CCI cross down 100 ', data.index[i], 'SELL'])), ignore_index=True)
                
        elif data['CCI ' + ticker][i] > 100 and first_cross == 1:
            print(ticker, '=> CCI cắt lên 100 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Chờ xem SELL')
            first_cross = 0
            df_CCI = df_CCI.append(dict(zip(df_CCI.columns,[ticker, 'CCI cross up 100 ', data.index[i], 'Wait to SELL'])), ignore_index=True)
            
        elif data['CCI ' + ticker][i] > -100 and data['CCI ' + ticker][i] < 0 and first_cross == 0:
            print(ticker, '=> CCI cắt lên -100 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 1
            df_CCI = df_CCI.append(dict(zip(df_CCI.columns,[ticker, 'CCI cross up -100 ', data.index[i], 'BUY'])), ignore_index=True)
           
        elif data['CCI ' + ticker][i] < -100 and first_cross == 1:
            print(ticker, '=> CCI cắt xuống -100 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Chờ xem BUY')
            first_cross = 0
            df_CCI = df_CCI.append(dict(zip(df_CCI.columns,[ticker, 'CCI cross down -100 ', data.index[i], 'Wait to BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_CCI.to_csv("CCI" +".txt", index=False)
print('Hoàn thành lưu CCI.txt')

ATVI => CCI cắt xuống 100 => Ngày: 12/04/2021 : Nên SELL
ATVI => CCI cắt xuống -100 => Ngày: 28/04/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 05/05/2021 : Nên BUY
ATVI => CCI cắt lên 100 => Ngày: 20/05/2021 : Chờ xem SELL
ATVI => CCI cắt xuống 100 => Ngày: 27/05/2021 : Nên SELL
ATVI => CCI cắt lên 100 => Ngày: 14/06/2021 : Chờ xem SELL
ATVI => CCI cắt xuống 100 => Ngày: 15/06/2021 : Nên SELL
ATVI => CCI cắt xuống -100 => Ngày: 16/06/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 24/06/2021 : Nên BUY
ATVI => CCI cắt xuống -100 => Ngày: 19/07/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 20/07/2021 : Nên BUY
ATVI => CCI cắt xuống -100 => Ngày: 22/07/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 23/07/2021 : Nên BUY
ATVI => CCI cắt xuống -100 => Ngày: 27/07/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 06/08/2021 : Nên BUY
ATVI => CCI cắt xuống -100 => Ngày: 07/09/2021 : Chờ xem BUY
ATVI => CCI cắt lên -100 => Ngày: 10/09/2021 : Nên BUY
ATVI => CCI cắt

AEP => CCI cắt lên 100 => Ngày: 20/08/2021 : Chờ xem SELL
AEP => CCI cắt xuống 100 => Ngày: 23/08/2021 : Nên SELL
AEP => CCI cắt lên 100 => Ngày: 02/09/2021 : Chờ xem SELL
AEP => CCI cắt xuống 100 => Ngày: 03/09/2021 : Nên SELL
AEP => CCI cắt xuống -100 => Ngày: 13/09/2021 : Chờ xem BUY
AEP => CCI cắt lên -100 => Ngày: 29/09/2021 : Nên BUY
AMGN => CCI cắt lên 100 => Ngày: 15/04/2021 : Chờ xem SELL
AMGN => CCI cắt xuống 100 => Ngày: 22/04/2021 : Nên SELL
AMGN => CCI cắt xuống -100 => Ngày: 28/04/2021 : Chờ xem BUY
AMGN => CCI cắt lên -100 => Ngày: 03/05/2021 : Nên BUY
AMGN => CCI cắt xuống -100 => Ngày: 27/05/2021 : Chờ xem BUY
AMGN => CCI cắt lên -100 => Ngày: 04/06/2021 : Nên BUY
AMGN => CCI cắt lên 100 => Ngày: 28/06/2021 : Chờ xem SELL
AMGN => CCI cắt xuống 100 => Ngày: 29/06/2021 : Nên SELL
AMGN => CCI cắt lên 100 => Ngày: 01/07/2021 : Chờ xem SELL
AMGN => CCI cắt xuống 100 => Ngày: 06/07/2021 : Nên SELL
AMGN => CCI cắt lên 100 => Ngày: 16/07/2021 : Chờ xem SELL
AMGN => CCI cắt xuố

TEAM => CCI cắt xuống 100 => Ngày: 26/07/2021 : Nên SELL
TEAM => CCI cắt lên 100 => Ngày: 30/07/2021 : Chờ xem SELL
TEAM => CCI cắt xuống 100 => Ngày: 09/08/2021 : Nên SELL
TEAM => CCI cắt lên 100 => Ngày: 26/08/2021 : Chờ xem SELL
TEAM => CCI cắt xuống 100 => Ngày: 09/09/2021 : Nên SELL
TEAM => CCI cắt lên 100 => Ngày: 15/09/2021 : Chờ xem SELL
TEAM => CCI cắt xuống 100 => Ngày: 20/09/2021 : Nên SELL
TEAM => CCI cắt lên 100 => Ngày: 21/09/2021 : Chờ xem SELL
TEAM => CCI cắt xuống 100 => Ngày: 22/09/2021 : Nên SELL
ADSK => CCI cắt lên 100 => Ngày: 12/04/2021 : Chờ xem SELL
ADSK => CCI cắt xuống 100 => Ngày: 14/04/2021 : Nên SELL
ADSK => CCI cắt lên 100 => Ngày: 15/04/2021 : Chờ xem SELL
ADSK => CCI cắt xuống 100 => Ngày: 16/04/2021 : Nên SELL
ADSK => CCI cắt xuống -100 => Ngày: 04/05/2021 : Chờ xem BUY
ADSK => CCI cắt lên -100 => Ngày: 07/05/2021 : Nên BUY
ADSK => CCI cắt xuống -100 => Ngày: 10/05/2021 : Chờ xem BUY
ADSK => CCI cắt lên -100 => Ngày: 14/05/2021 : Nên BUY
ADSK => CCI cắt

CDNS => CCI cắt xuống 100 => Ngày: 16/06/2021 : Nên SELL
CDNS => CCI cắt lên 100 => Ngày: 17/06/2021 : Chờ xem SELL
CDNS => CCI cắt xuống 100 => Ngày: 01/07/2021 : Nên SELL
CDNS => CCI cắt lên 100 => Ngày: 21/07/2021 : Chờ xem SELL
CDNS => CCI cắt xuống 100 => Ngày: 06/08/2021 : Nên SELL
CDNS => CCI cắt lên 100 => Ngày: 20/08/2021 : Chờ xem SELL
CDNS => CCI cắt xuống 100 => Ngày: 07/09/2021 : Nên SELL
CDNS => CCI cắt xuống -100 => Ngày: 27/09/2021 : Chờ xem BUY
CDNS => CCI cắt lên -100 => Ngày: 07/10/2021 : Nên BUY
CDW => CCI cắt lên 100 => Ngày: 12/04/2021 : Chờ xem SELL
CDW => CCI cắt xuống 100 => Ngày: 13/04/2021 : Nên SELL
CDW => CCI cắt lên 100 => Ngày: 14/04/2021 : Chờ xem SELL
CDW => CCI cắt xuống 100 => Ngày: 20/04/2021 : Nên SELL
CDW => CCI cắt xuống -100 => Ngày: 05/05/2021 : Chờ xem BUY
CDW => CCI cắt lên -100 => Ngày: 07/05/2021 : Nên BUY
CDW => CCI cắt xuống -100 => Ngày: 10/05/2021 : Chờ xem BUY
CDW => CCI cắt lên -100 => Ngày: 14/05/2021 : Nên BUY
CDW => CCI cắt lên 100 

CSCO => CCI cắt lên -100 => Ngày: 23/09/2021 : Nên BUY
CSCO => CCI cắt xuống -100 => Ngày: 30/09/2021 : Chờ xem BUY
CSCO => CCI cắt lên -100 => Ngày: 01/10/2021 : Nên BUY
CSCO => CCI cắt xuống -100 => Ngày: 06/10/2021 : Chờ xem BUY
CSCO => CCI cắt lên -100 => Ngày: 07/10/2021 : Nên BUY
CTSH => CCI cắt lên 100 => Ngày: 29/04/2021 : Chờ xem SELL
CTSH => CCI cắt xuống 100 => Ngày: 30/04/2021 : Nên SELL
CTSH => CCI cắt lên 100 => Ngày: 03/05/2021 : Chờ xem SELL
CTSH => CCI cắt xuống 100 => Ngày: 05/05/2021 : Nên SELL
CTSH => CCI cắt xuống -100 => Ngày: 06/05/2021 : Chờ xem BUY
CTSH => CCI cắt lên -100 => Ngày: 18/05/2021 : Nên BUY
CTSH => CCI cắt xuống -100 => Ngày: 16/06/2021 : Chờ xem BUY
CTSH => CCI cắt lên -100 => Ngày: 22/06/2021 : Nên BUY
CTSH => CCI cắt xuống -100 => Ngày: 08/07/2021 : Chờ xem BUY
CTSH => CCI cắt lên -100 => Ngày: 09/07/2021 : Nên BUY
CTSH => CCI cắt xuống -100 => Ngày: 19/07/2021 : Chờ xem BUY
CTSH => CCI cắt lên -100 => Ngày: 21/07/2021 : Nên BUY
CTSH => CCI cắt l

...

NVDA => CCI cắt lên -100 => Ngày: 07/05/2021 : Nên BUY
NVDA => CCI cắt xuống -100 => Ngày: 10/05/2021 : Chờ xem BUY
NVDA => CCI cắt lên -100 => Ngày: 14/05/2021 : Nên BUY
NVDA => CCI cắt lên 100 => Ngày: 24/05/2021 : Chờ xem SELL
NVDA => CCI cắt xuống 100 => Ngày: 27/05/2021 : Nên SELL
NVDA => CCI cắt lên 100 => Ngày: 28/05/2021 : Chờ xem SELL
NVDA => CCI cắt xuống 100 => Ngày: 09/06/2021 : Nên SELL
NVDA => CCI cắt lên 100 => Ngày: 17/06/2021 : Chờ xem SELL
NVDA => CCI cắt xuống 100 => Ngày: 21/06/2021 : Nên SELL
NVDA => CCI cắt lên 100 => Ngày: 23/06/2021 : Chờ xem SELL
NVDA => CCI cắt xuống 100 => Ngày: 25/06/2021 : Nên SELL
NVDA => CCI cắt lên 100 => Ngày: 28/06/2021 : Chờ xem SELL
NVDA => CCI cắt xuống 100 => Ngày: 07/07/2021 : Nên SELL
NVDA => CCI cắt xuống -100 => Ngày: 16/07/2021 : Chờ xem BUY
NVDA => CCI cắt lên -100 => Ngày: 19/07/2021 : Nên BUY
NVDA => CCI cắt xuống -100 => Ngày: 20/07/2021 : Chờ xem BUY
NVDA => CCI cắt lên -100 => Ngày: 21/07/2021 : Nên BUY
NVDA => CCI cắt l

PYPL => CCI cắt xuống 100 => Ngày: 03/06/2021 : Nên SELL
PYPL => CCI cắt lên 100 => Ngày: 11/06/2021 : Chờ xem SELL
PYPL => CCI cắt xuống 100 => Ngày: 16/06/2021 : Nên SELL
PYPL => CCI cắt lên 100 => Ngày: 17/06/2021 : Chờ xem SELL
PYPL => CCI cắt xuống 100 => Ngày: 30/06/2021 : Nên SELL
PYPL => CCI cắt lên 100 => Ngày: 12/07/2021 : Chờ xem SELL
PYPL => CCI cắt xuống 100 => Ngày: 14/07/2021 : Nên SELL
PYPL => CCI cắt lên 100 => Ngày: 22/07/2021 : Chờ xem SELL
PYPL => CCI cắt xuống 100 => Ngày: 27/07/2021 : Nên SELL
PYPL => CCI cắt xuống -100 => Ngày: 29/07/2021 : Chờ xem BUY
PYPL => CCI cắt lên -100 => Ngày: 05/08/2021 : Nên BUY
PYPL => CCI cắt lên 100 => Ngày: 30/08/2021 : Chờ xem SELL
PYPL => CCI cắt xuống 100 => Ngày: 08/09/2021 : Nên SELL
PYPL => CCI cắt xuống -100 => Ngày: 20/09/2021 : Chờ xem BUY
PYPL => CCI cắt lên -100 => Ngày: 23/09/2021 : Nên BUY
PYPL => CCI cắt xuống -100 => Ngày: 28/09/2021 : Chờ xem BUY
PYPL => CCI cắt lên -100 => Ngày: 01/10/2021 : Nên BUY
PYPL => CCI cắt

ROST => CCI cắt xuống 100 => Ngày: 28/06/2021 : Nên SELL
ROST => CCI cắt lên 100 => Ngày: 30/06/2021 : Chờ xem SELL
ROST => CCI cắt xuống 100 => Ngày: 07/07/2021 : Nên SELL
ROST => CCI cắt xuống -100 => Ngày: 19/07/2021 : Chờ xem BUY
ROST => CCI cắt lên -100 => Ngày: 21/07/2021 : Nên BUY
ROST => CCI cắt lên 100 => Ngày: 11/08/2021 : Chờ xem SELL
ROST => CCI cắt xuống 100 => Ngày: 13/08/2021 : Nên SELL
ROST => CCI cắt xuống -100 => Ngày: 27/08/2021 : Chờ xem BUY
ROST => CCI cắt lên -100 => Ngày: 09/09/2021 : Nên BUY
ROST => CCI cắt xuống -100 => Ngày: 10/09/2021 : Chờ xem BUY
ROST => CCI cắt lên -100 => Ngày: 13/09/2021 : Nên BUY
ROST => CCI cắt xuống -100 => Ngày: 30/09/2021 : Chờ xem BUY
ROST => CCI cắt lên -100 => Ngày: 13/10/2021 : Nên BUY
SGEN => CCI cắt lên 100 => Ngày: 15/04/2021 : Chờ xem SELL
SGEN => CCI cắt xuống 100 => Ngày: 16/04/2021 : Nên SELL
SGEN => CCI cắt xuống -100 => Ngày: 29/04/2021 : Chờ xem BUY
SGEN => CCI cắt lên -100 => Ngày: 30/04/2021 : Nên BUY
SGEN => CCI cắt

TMUS => CCI cắt xuống 100 => Ngày: 19/04/2021 : Nên SELL
TMUS => CCI cắt xuống -100 => Ngày: 04/05/2021 : Chờ xem BUY
TMUS => CCI cắt xuống 100 => Ngày: 12/05/2021 : Nên SELL
TMUS => CCI cắt lên 100 => Ngày: 13/05/2021 : Chờ xem SELL
TMUS => CCI cắt xuống 100 => Ngày: 18/05/2021 : Nên SELL
TMUS => CCI cắt lên 100 => Ngày: 28/05/2021 : Chờ xem SELL
TMUS => CCI cắt xuống 100 => Ngày: 01/06/2021 : Nên SELL
TMUS => CCI cắt lên 100 => Ngày: 04/06/2021 : Chờ xem SELL
TMUS => CCI cắt xuống 100 => Ngày: 11/06/2021 : Nên SELL
TMUS => CCI cắt lên 100 => Ngày: 07/07/2021 : Chờ xem SELL
TMUS => CCI cắt xuống 100 => Ngày: 09/07/2021 : Nên SELL
TMUS => CCI cắt lên 100 => Ngày: 13/07/2021 : Chờ xem SELL
TMUS => CCI cắt lên -100 => Ngày: 19/07/2021 : Nên BUY
TMUS => CCI cắt xuống -100 => Ngày: 05/08/2021 : Chờ xem BUY
TMUS => CCI cắt lên -100 => Ngày: 06/08/2021 : Nên BUY
TMUS => CCI cắt xuống -100 => Ngày: 16/08/2021 : Chờ xem BUY
TMUS => CCI cắt lên -100 => Ngày: 20/08/2021 : Nên BUY
TMUS => CCI cắt

VRSK => CCI cắt xuống 100 => Ngày: 27/09/2021 : Nên SELL
VRSK => CCI cắt xuống -100 => Ngày: 01/10/2021 : Chờ xem BUY
VRSK => CCI cắt lên -100 => Ngày: 05/10/2021 : Nên BUY
VRSK => CCI cắt lên 100 => Ngày: 14/10/2021 : Chờ xem SELL
VRTX => CCI cắt lên -100 => Ngày: 12/04/2021 : Nên BUY
VRTX => CCI cắt lên 100 => Ngày: 14/04/2021 : Chờ xem SELL
VRTX => CCI cắt xuống 100 => Ngày: 21/04/2021 : Nên SELL
VRTX => CCI cắt xuống -100 => Ngày: 25/05/2021 : Chờ xem BUY
VRTX => CCI cắt lên -100 => Ngày: 28/05/2021 : Nên BUY
VRTX => CCI cắt xuống -100 => Ngày: 11/06/2021 : Chờ xem BUY
VRTX => CCI cắt lên -100 => Ngày: 21/06/2021 : Nên BUY
VRTX => CCI cắt lên 100 => Ngày: 28/07/2021 : Chờ xem SELL
VRTX => CCI cắt xuống 100 => Ngày: 29/07/2021 : Nên SELL
VRTX => CCI cắt xuống -100 => Ngày: 12/08/2021 : Chờ xem BUY
VRTX => CCI cắt lên -100 => Ngày: 17/08/2021 : Nên BUY
VRTX => CCI cắt xuống -100 => Ngày: 08/09/2021 : Chờ xem BUY
VRTX => CCI cắt lên -100 => Ngày: 13/09/2021 : Nên BUY
VRTX => CCI cắt x

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# An toàn RSI: Thị trường giảm mạnh -> cắt lên 30 BUY, cắt xuống 70 SELL
### Thông báo: Biến động giá trong thời gian dài
### Nếu RSI + CCI + MACD trong 2,3 ngày thì tín hiệu quá tốt
### Lọc trong vòng 1 tháng

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [5]:
first_cross = 0
df_RSI = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])

#Loop throught the lenght of the data set
for ticker in tickers_list:  
    for i in range(0, len(data)):
        if data['RSI ' + ticker][i] < 70 and data['RSI ' + ticker][i] > 50 and first_cross == 0:
            print(ticker, '=> RSI cắt xuống 70 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[ticker, 'RSI cross down 70 ', data.index[i], 'SELL'])), ignore_index=True)
            
        elif data['RSI ' + ticker][i] > 70 and first_cross == 1:
            print(ticker, '=> RSI cắt lên 70 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Chờ xem SELL')
            first_cross = 0  
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[ticker, 'RSI cross up 70 ', data.index[i], 'Wait to SELL'])), ignore_index=True)
                
        elif data['RSI ' + ticker][i] > 30 and data['RSI ' + ticker][i] < 50 and first_cross == 0:
            print(ticker, '=> RSI cắt lên 30 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 1                
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[ticker, 'RSI cross up 30 ', data.index[i], 'BUY'])), ignore_index=True)
                
        elif data['RSI ' + ticker][i] < 30 and first_cross == 1:
            print(ticker, '=> RSI cắt xuống 30 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Chờ xem BUY')
            first_cross = 0               
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[ticker, 'RSI cross down 30 ', data.index[i], 'Wait to BUY'])), ignore_index=True)

#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_RSI.to_csv("RSI" +".txt", index=False)
print('Hoàn thành lưu RSI.txt')

ATVI => RSI cắt xuống 70 => Ngày: 12/04/2021 : Nên SELL
ATVI => RSI cắt xuống 30 => Ngày: 27/07/2021 : Chờ xem BUY
ATVI => RSI cắt lên 30 => Ngày: 28/07/2021 : Nên BUY
ATVI => RSI cắt xuống 30 => Ngày: 29/07/2021 : Chờ xem BUY
ATVI => RSI cắt lên 30 => Ngày: 04/08/2021 : Nên BUY
ATVI => RSI cắt xuống 30 => Ngày: 05/08/2021 : Chờ xem BUY
ATVI => RSI cắt lên 30 => Ngày: 06/08/2021 : Nên BUY
ATVI => RSI cắt xuống 30 => Ngày: 21/09/2021 : Chờ xem BUY
ATVI => RSI cắt lên 30 => Ngày: 23/09/2021 : Nên BUY
ADBE => RSI cắt lên 70 => Ngày: 13/04/2021 : Chờ xem SELL
ADBE => RSI cắt xuống 70 => Ngày: 14/04/2021 : Nên SELL
ADBE => RSI cắt lên 70 => Ngày: 15/04/2021 : Chờ xem SELL
ADBE => RSI cắt xuống 70 => Ngày: 19/04/2021 : Nên SELL
ADBE => RSI cắt lên 70 => Ngày: 11/06/2021 : Chờ xem SELL
ADBE => RSI cắt xuống 70 => Ngày: 15/06/2021 : Nên SELL
ADBE => RSI cắt lên 70 => Ngày: 18/06/2021 : Chờ xem SELL
ADBE => RSI cắt xuống 70 => Ngày: 19/07/2021 : Nên SELL
ADBE => RSI cắt lên 70 => Ngày: 20/07/20

CDNS => RSI cắt xuống 70 => Ngày: 17/09/2021 : Nên SELL
CDW => RSI cắt lên 70 => Ngày: 13/04/2021 : Chờ xem SELL
CDW => RSI cắt xuống 70 => Ngày: 20/04/2021 : Nên SELL
CDW => RSI cắt lên 70 => Ngày: 21/04/2021 : Chờ xem SELL
CDW => RSI cắt xuống 70 => Ngày: 22/04/2021 : Nên SELL
CDW => RSI cắt lên 70 => Ngày: 23/04/2021 : Chờ xem SELL
CDW => RSI cắt xuống 70 => Ngày: 28/04/2021 : Nên SELL
CDW => RSI cắt lên 70 => Ngày: 04/08/2021 : Chờ xem SELL
CDW => RSI cắt xuống 70 => Ngày: 18/08/2021 : Nên SELL
CDW => RSI cắt lên 70 => Ngày: 23/08/2021 : Chờ xem SELL
CDW => RSI cắt xuống 70 => Ngày: 07/09/2021 : Nên SELL
CDW => RSI cắt xuống 30 => Ngày: 06/10/2021 : Chờ xem BUY
CDW => RSI cắt lên 30 => Ngày: 07/10/2021 : Nên BUY
CDW => RSI cắt xuống 30 => Ngày: 12/10/2021 : Chờ xem BUY
CDW => RSI cắt lên 30 => Ngày: 13/10/2021 : Nên BUY
CERN => RSI cắt xuống 30 => Ngày: 14/09/2021 : Chờ xem BUY
CERN => RSI cắt lên 30 => Ngày: 15/09/2021 : Nên BUY
CERN => RSI cắt xuống 30 => Ngày: 27/09/2021 : Chờ x

EXC => RSI cắt xuống 70 => Ngày: 23/08/2021 : Nên SELL
EXC => RSI cắt lên 70 => Ngày: 02/09/2021 : Chờ xem SELL
EXC => RSI cắt xuống 70 => Ngày: 03/09/2021 : Nên SELL
FB => RSI cắt lên 70 => Ngày: 29/04/2021 : Chờ xem SELL
FB => RSI cắt xuống 70 => Ngày: 30/04/2021 : Nên SELL
FB => RSI cắt xuống 30 => Ngày: 04/10/2021 : Chờ xem BUY
FB => RSI cắt lên 30 => Ngày: 05/10/2021 : Nên BUY
FAST => RSI cắt lên 70 => Ngày: 05/05/2021 : Chờ xem SELL
FAST => RSI cắt xuống 70 => Ngày: 06/05/2021 : Nên SELL
FAST => RSI cắt lên 70 => Ngày: 03/08/2021 : Chờ xem SELL
FAST => RSI cắt xuống 70 => Ngày: 04/08/2021 : Nên SELL
FISV => RSI cắt xuống 30 => Ngày: 12/05/2021 : Chờ xem BUY
FISV => RSI cắt lên 30 => Ngày: 13/05/2021 : Nên BUY
FOXA => RSI cắt lên 70 => Ngày: 01/10/2021 : Chờ xem SELL
FOXA => RSI cắt xuống 70 => Ngày: 15/10/2021 : Nên SELL
FOX => RSI cắt lên 70 => Ngày: 01/10/2021 : Chờ xem SELL
FOX => RSI cắt xuống 70 => Ngày: 15/10/2021 : Nên SELL
GILD => RSI cắt lên 70 => Ngày: 17/08/2021 : Chờ 

ORLY => RSI cắt lên 70 => Ngày: 29/04/2021 : Chờ xem SELL
ORLY => RSI cắt xuống 70 => Ngày: 12/05/2021 : Nên SELL
ORLY => RSI cắt lên 70 => Ngày: 28/06/2021 : Chờ xem SELL
ORLY => RSI cắt xuống 70 => Ngày: 28/07/2021 : Nên SELL
OKTA => RSI cắt lên 70 => Ngày: 02/09/2021 : Chờ xem SELL
OKTA => RSI cắt xuống 70 => Ngày: 03/09/2021 : Nên SELL
PCAR => RSI cắt xuống 30 => Ngày: 04/08/2021 : Chờ xem BUY
PCAR => RSI cắt lên 30 => Ngày: 11/08/2021 : Nên BUY
PAYX => RSI cắt lên 70 => Ngày: 01/07/2021 : Chờ xem SELL
PAYX => RSI cắt xuống 70 => Ngày: 19/07/2021 : Nên SELL
PAYX => RSI cắt lên 70 => Ngày: 30/07/2021 : Chờ xem SELL
PAYX => RSI cắt xuống 70 => Ngày: 02/08/2021 : Nên SELL
PAYX => RSI cắt lên 70 => Ngày: 03/08/2021 : Chờ xem SELL
PAYX => RSI cắt xuống 70 => Ngày: 04/08/2021 : Nên SELL
PAYX => RSI cắt lên 70 => Ngày: 05/08/2021 : Chờ xem SELL
PAYX => RSI cắt xuống 70 => Ngày: 09/08/2021 : Nên SELL
PAYX => RSI cắt lên 70 => Ngày: 10/08/2021 : Chờ xem SELL
PAYX => RSI cắt xuống 70 => Ngày

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Bình thường MACD
### Lọc trong vòng 3 tháng


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [6]:
first_cross = 0
df_MACD = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
    
#Loop throught the lenght of the data set
for ticker in tickers_list:   
    for i in range(0, len(data)):
        if data['MACD ' + ticker][i] < data['Signal ' + ticker][i] and first_cross == 0:
            print(ticker, '=> MACD cắt xuống Signal => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1                
            df_MACD = df_MACD.append(dict(zip(df_MACD.columns,[ticker, ' MACD cắt xuống Signal ', data.index[i], 'SELL'])), ignore_index=True)
                
        elif data['MACD ' + ticker][i] > data['Signal ' + ticker][i] and first_cross == 1:
            print(ticker, '=> MACD cắt lên Signal => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 0                
            df_MACD = df_MACD.append(dict(zip(df_MACD.columns,[ticker, ' MACD cắt lên Signal ', data.index[i], 'BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_MACD.to_csv("MACD" +".txt", index=False)
print('Hoàn thành lưu MACD.txt')

ATVI => MACD cắt xuống Signal => Ngày: 21/04/2021 : Nên SELL
ATVI => MACD cắt lên Signal => Ngày: 10/05/2021 : Nên BUY
ATVI => MACD cắt xuống Signal => Ngày: 16/06/2021 : Nên SELL
ATVI => MACD cắt lên Signal => Ngày: 30/06/2021 : Nên BUY
ATVI => MACD cắt xuống Signal => Ngày: 08/07/2021 : Nên SELL
ATVI => MACD cắt lên Signal => Ngày: 11/08/2021 : Nên BUY
ATVI => MACD cắt xuống Signal => Ngày: 09/09/2021 : Nên SELL
ATVI => MACD cắt lên Signal => Ngày: 16/09/2021 : Nên BUY
ATVI => MACD cắt xuống Signal => Ngày: 20/09/2021 : Nên SELL
ATVI => MACD cắt lên Signal => Ngày: 29/09/2021 : Nên BUY
ADBE => MACD cắt xuống Signal => Ngày: 26/04/2021 : Nên SELL
ADBE => MACD cắt lên Signal => Ngày: 24/05/2021 : Nên BUY
ADBE => MACD cắt xuống Signal => Ngày: 12/07/2021 : Nên SELL
ADBE => MACD cắt lên Signal => Ngày: 23/08/2021 : Nên BUY
ADBE => MACD cắt xuống Signal => Ngày: 07/09/2021 : Nên SELL
ADBE => MACD cắt lên Signal => Ngày: 13/10/2021 : Nên BUY
AMD => MACD cắt xuống Signal => Ngày: 03/05/2021

ADSK => MACD cắt xuống Signal => Ngày: 22/04/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 26/04/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 30/04/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 24/05/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 03/06/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 04/06/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 07/06/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 21/06/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 16/07/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 22/07/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 16/08/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 25/08/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 26/08/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 23/09/2021 : Nên BUY
ADSK => MACD cắt xuống Signal => Ngày: 05/10/2021 : Nên SELL
ADSK => MACD cắt lên Signal => Ngày: 07/10/2021 : Nên BUY
ADP => MACD cắt xuống Signal => Ngày: 12/04/2021

CSCO => MACD cắt lên Signal => Ngày: 07/07/2021 : Nên BUY
CSCO => MACD cắt xuống Signal => Ngày: 17/08/2021 : Nên SELL
CSCO => MACD cắt lên Signal => Ngày: 19/08/2021 : Nên BUY
CSCO => MACD cắt xuống Signal => Ngày: 07/09/2021 : Nên SELL
CSCO => MACD cắt lên Signal => Ngày: 14/10/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 20/04/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 03/05/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 06/05/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 28/05/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 18/06/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 21/06/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 19/07/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 23/07/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 30/08/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 27/09/2021 : Nên BUY
CTSH => MACD cắt xuống Signal => Ngày: 28/09/2021 : Nên SELL
CTSH => MACD cắt lên Signal => Ngày: 14/10/2021 

FB => MACD cắt xuống Signal => Ngày: 08/07/2021 : Nên SELL
FB => MACD cắt lên Signal => Ngày: 23/07/2021 : Nên BUY
FB => MACD cắt xuống Signal => Ngày: 02/08/2021 : Nên SELL
FB => MACD cắt lên Signal => Ngày: 27/08/2021 : Nên BUY
FB => MACD cắt xuống Signal => Ngày: 13/09/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 12/04/2021 : Nên BUY
FAST => MACD cắt xuống Signal => Ngày: 13/04/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 21/04/2021 : Nên BUY
FAST => MACD cắt xuống Signal => Ngày: 11/05/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 30/06/2021 : Nên BUY
FAST => MACD cắt xuống Signal => Ngày: 10/08/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 11/08/2021 : Nên BUY
FAST => MACD cắt xuống Signal => Ngày: 12/08/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 27/09/2021 : Nên BUY
FAST => MACD cắt xuống Signal => Ngày: 28/09/2021 : Nên SELL
FAST => MACD cắt lên Signal => Ngày: 08/10/2021 : Nên BUY
FISV => MACD cắt xuống Signal => Ngày: 12/04/2021 : Nên SE

JD => MACD cắt xuống Signal => Ngày: 10/05/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 20/05/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 11/06/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 21/06/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 07/07/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 15/07/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 19/07/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 22/07/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 23/07/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 09/08/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 16/08/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 24/08/2021 : Nên BUY
JD => MACD cắt xuống Signal => Ngày: 15/09/2021 : Nên SELL
JD => MACD cắt lên Signal => Ngày: 08/10/2021 : Nên BUY
KDP => MACD cắt xuống Signal => Ngày: 12/04/2021 : Nên SELL
KDP => MACD cắt lên Signal => Ngày: 19/04/2021 : Nên BUY
KDP => MACD cắt xuống Signal => Ngày: 22/04/2021 : Nên SELL
KDP => MACD cắt lê

MDLZ => MACD cắt lên Signal => Ngày: 28/04/2021 : Nên BUY
MDLZ => MACD cắt xuống Signal => Ngày: 28/05/2021 : Nên SELL
MDLZ => MACD cắt lên Signal => Ngày: 09/07/2021 : Nên BUY
MDLZ => MACD cắt xuống Signal => Ngày: 29/07/2021 : Nên SELL
MDLZ => MACD cắt lên Signal => Ngày: 16/08/2021 : Nên BUY
MDLZ => MACD cắt xuống Signal => Ngày: 24/08/2021 : Nên SELL
MDLZ => MACD cắt lên Signal => Ngày: 22/09/2021 : Nên BUY
MDLZ => MACD cắt xuống Signal => Ngày: 27/09/2021 : Nên SELL
MDLZ => MACD cắt lên Signal => Ngày: 06/10/2021 : Nên BUY
MNST => MACD cắt xuống Signal => Ngày: 27/04/2021 : Nên SELL
MNST => MACD cắt lên Signal => Ngày: 25/05/2021 : Nên BUY
MNST => MACD cắt xuống Signal => Ngày: 15/06/2021 : Nên SELL
MNST => MACD cắt lên Signal => Ngày: 13/07/2021 : Nên BUY
MNST => MACD cắt xuống Signal => Ngày: 03/08/2021 : Nên SELL
MNST => MACD cắt lên Signal => Ngày: 06/08/2021 : Nên BUY
MNST => MACD cắt xuống Signal => Ngày: 20/08/2021 : Nên SELL
NTES => MACD cắt lên Signal => Ngày: 12/04/2021 

QCOM => MACD cắt xuống Signal => Ngày: 27/07/2021 : Nên SELL
QCOM => MACD cắt lên Signal => Ngày: 29/07/2021 : Nên BUY
QCOM => MACD cắt xuống Signal => Ngày: 09/08/2021 : Nên SELL
QCOM => MACD cắt lên Signal => Ngày: 01/09/2021 : Nên BUY
QCOM => MACD cắt xuống Signal => Ngày: 07/09/2021 : Nên SELL
QCOM => MACD cắt lên Signal => Ngày: 14/10/2021 : Nên BUY
REGN => MACD cắt xuống Signal => Ngày: 28/04/2021 : Nên SELL
REGN => MACD cắt lên Signal => Ngày: 10/05/2021 : Nên BUY
REGN => MACD cắt xuống Signal => Ngày: 24/05/2021 : Nên SELL
REGN => MACD cắt lên Signal => Ngày: 07/06/2021 : Nên BUY
REGN => MACD cắt xuống Signal => Ngày: 08/06/2021 : Nên SELL
REGN => MACD cắt lên Signal => Ngày: 09/06/2021 : Nên BUY
REGN => MACD cắt xuống Signal => Ngày: 19/07/2021 : Nên SELL
REGN => MACD cắt lên Signal => Ngày: 06/08/2021 : Nên BUY
REGN => MACD cắt xuống Signal => Ngày: 07/09/2021 : Nên SELL
ROST => MACD cắt lên Signal => Ngày: 12/04/2021 : Nên BUY
ROST => MACD cắt xuống Signal => Ngày: 26/04/202

VRSK => MACD cắt lên Signal => Ngày: 19/08/2021 : Nên BUY
VRSK => MACD cắt xuống Signal => Ngày: 14/09/2021 : Nên SELL
VRSK => MACD cắt lên Signal => Ngày: 11/10/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 12/04/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 13/04/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 27/04/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 17/05/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 24/05/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 08/06/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 11/06/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 28/06/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 10/08/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 24/08/2021 : Nên BUY
VRTX => MACD cắt xuống Signal => Ngày: 07/09/2021 : Nên SELL
VRTX => MACD cắt lên Signal => Ngày: 11/10/2021 : Nên BUY
WBA => MACD cắt xuống Signal => Ngày: 12/04/2021 : Nên SELL
WBA => MACD cắt lên Signal => Ngày: 06/05/2021 : 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

# Theo trend EMA12 & EMA36 
### Lọc trong vòng 3 tháng

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [7]:
first_cross = 0
df_EMA12_EMA36 = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
    
#Loop throught the lenght of the data set
for ticker in tickers_list:  
    for i in range(0, len(data)):
        if data['EMA12 ' + ticker][i] < data['EMA36 ' + ticker][i] and first_cross == 0:
            print(ticker, '=> EMA12 cắt xuống EMA36 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1                
            df_EMA12_EMA36 = df_EMA12_EMA36.append(dict(zip(df_EMA12_EMA36.columns,[ticker, ' EMA12 cắt xuống EMA36 ', data.index[i], 'SELL'])), ignore_index=True)
                
        elif data['EMA12 ' + ticker][i] > data['EMA36 ' + ticker][i] and first_cross == 1:
            print(ticker, '=> EMA12 cắt lên EMA36 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 0                
            df_EMA12_EMA36 = df_EMA12_EMA36.append(dict(zip(df_EMA12_EMA36.columns,[ticker, ' EMA12 cắt lên EMA36 ', data.index[i], 'BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_EMA12_EMA36.to_csv("EMA12 & EMA36" +".txt", index=False)
print('Hoàn thành lưu EMA12 & EMA36.txt')

ATVI => EMA12 cắt xuống EMA36 => Ngày: 28/04/2021 : Nên SELL
ATVI => EMA12 cắt lên EMA36 => Ngày: 20/05/2021 : Nên BUY
ATVI => EMA12 cắt xuống EMA36 => Ngày: 21/06/2021 : Nên SELL
ADBE => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
ADBE => EMA12 cắt xuống EMA36 => Ngày: 12/05/2021 : Nên SELL
ADBE => EMA12 cắt lên EMA36 => Ngày: 25/05/2021 : Nên BUY
ADBE => EMA12 cắt xuống EMA36 => Ngày: 27/09/2021 : Nên SELL
AMD => EMA12 cắt lên EMA36 => Ngày: 27/04/2021 : Nên BUY
AMD => EMA12 cắt xuống EMA36 => Ngày: 04/05/2021 : Nên SELL
AMD => EMA12 cắt lên EMA36 => Ngày: 04/06/2021 : Nên BUY
AMD => EMA12 cắt xuống EMA36 => Ngày: 29/09/2021 : Nên SELL
AMD => EMA12 cắt lên EMA36 => Ngày: 12/10/2021 : Nên BUY
ALGN => EMA12 cắt xuống EMA36 => Ngày: 13/05/2021 : Nên SELL
ALGN => EMA12 cắt lên EMA36 => Ngày: 21/05/2021 : Nên BUY
ALGN => EMA12 cắt xuống EMA36 => Ngày: 01/10/2021 : Nên SELL
GOOGL => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
GOOGL => EMA12 cắt xuống EMA36 => Ngày: 30/09/2021 :

DOCU => EMA12 cắt xuống EMA36 => Ngày: 13/09/2021 : Nên SELL
DLTR => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
DLTR => EMA12 cắt xuống EMA36 => Ngày: 13/05/2021 : Nên SELL
DLTR => EMA12 cắt lên EMA36 => Ngày: 13/08/2021 : Nên BUY
DLTR => EMA12 cắt xuống EMA36 => Ngày: 27/08/2021 : Nên SELL
DLTR => EMA12 cắt lên EMA36 => Ngày: 06/10/2021 : Nên BUY
EBAY => EMA12 cắt xuống EMA36 => Ngày: 30/04/2021 : Nên SELL
EBAY => EMA12 cắt lên EMA36 => Ngày: 18/05/2021 : Nên BUY
EBAY => EMA12 cắt xuống EMA36 => Ngày: 19/05/2021 : Nên SELL
EBAY => EMA12 cắt lên EMA36 => Ngày: 21/05/2021 : Nên BUY
EBAY => EMA12 cắt xuống EMA36 => Ngày: 30/09/2021 : Nên SELL
EBAY => EMA12 cắt lên EMA36 => Ngày: 11/10/2021 : Nên BUY
EA => EMA12 cắt xuống EMA36 => Ngày: 18/05/2021 : Nên SELL
EA => EMA12 cắt lên EMA36 => Ngày: 20/05/2021 : Nên BUY
EA => EMA12 cắt xuống EMA36 => Ngày: 24/06/2021 : Nên SELL
EA => EMA12 cắt lên EMA36 => Ngày: 30/06/2021 : Nên BUY
EA => EMA12 cắt xuống EMA36 => Ngày: 09/07/2021 : Nên SE

MCHP => EMA12 cắt xuống EMA36 => Ngày: 06/10/2021 : Nên SELL
MU => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
MU => EMA12 cắt xuống EMA36 => Ngày: 26/04/2021 : Nên SELL
MU => EMA12 cắt lên EMA36 => Ngày: 27/04/2021 : Nên BUY
MU => EMA12 cắt xuống EMA36 => Ngày: 28/04/2021 : Nên SELL
MU => EMA12 cắt lên EMA36 => Ngày: 06/08/2021 : Nên BUY
MU => EMA12 cắt xuống EMA36 => Ngày: 10/08/2021 : Nên SELL
MSFT => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
MSFT => EMA12 cắt xuống EMA36 => Ngày: 13/05/2021 : Nên SELL
MSFT => EMA12 cắt lên EMA36 => Ngày: 25/05/2021 : Nên BUY
MSFT => EMA12 cắt xuống EMA36 => Ngày: 29/09/2021 : Nên SELL
MSFT => EMA12 cắt lên EMA36 => Ngày: 14/10/2021 : Nên BUY
MRNA => EMA12 cắt xuống EMA36 => Ngày: 12/04/2021 : Nên SELL
MRNA => EMA12 cắt lên EMA36 => Ngày: 14/04/2021 : Nên BUY
MRNA => EMA12 cắt xuống EMA36 => Ngày: 04/10/2021 : Nên SELL
MDLZ => EMA12 cắt lên EMA36 => Ngày: 12/04/2021 : Nên BUY
MDLZ => EMA12 cắt xuống EMA36 => Ngày: 06/08/2021 : Nên SELL

TXN => EMA12 cắt xuống EMA36 => Ngày: 12/05/2021 : Nên SELL
TXN => EMA12 cắt lên EMA36 => Ngày: 24/05/2021 : Nên BUY
TXN => EMA12 cắt xuống EMA36 => Ngày: 28/07/2021 : Nên SELL
TXN => EMA12 cắt lên EMA36 => Ngày: 29/07/2021 : Nên BUY
TXN => EMA12 cắt xuống EMA36 => Ngày: 18/08/2021 : Nên SELL
TXN => EMA12 cắt lên EMA36 => Ngày: 30/08/2021 : Nên BUY
TCOM => EMA12 cắt xuống EMA36 => Ngày: 12/04/2021 : Nên SELL
TCOM => EMA12 cắt lên EMA36 => Ngày: 28/04/2021 : Nên BUY
TCOM => EMA12 cắt xuống EMA36 => Ngày: 11/05/2021 : Nên SELL
TCOM => EMA12 cắt lên EMA36 => Ngày: 19/05/2021 : Nên BUY
TCOM => EMA12 cắt xuống EMA36 => Ngày: 09/06/2021 : Nên SELL
TCOM => EMA12 cắt lên EMA36 => Ngày: 01/09/2021 : Nên BUY
TCOM => EMA12 cắt xuống EMA36 => Ngày: 22/09/2021 : Nên SELL
TCOM => EMA12 cắt lên EMA36 => Ngày: 23/09/2021 : Nên BUY
VRSN => EMA12 cắt xuống EMA36 => Ngày: 28/07/2021 : Nên SELL
VRSN => EMA12 cắt lên EMA36 => Ngày: 08/09/2021 : Nên BUY
VRSN => EMA12 cắt xuống EMA36 => Ngày: 22/09/2021 : Nê

# MFI

In [8]:
import warnings
warnings.filterwarnings('ignore')

#Create a new data frame
df_MFI = pd.DataFrame()
df_MFI = data[14:]
for ticker in tickers_list:
    period = 14
    #Calculate the money flow 
    money_flow = data['TP ' + ticker] * data['Volume ' + ticker]

    #Get all of the positive and negative money flows
    positive_flow = []
    negative_flow = []
    #Loop through the typical price
    for i in range(1, len(data['TP ' + ticker])):
        if data['TP ' + ticker][i] > data['TP ' + ticker][i-1]:
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
        elif data['TP ' + ticker][i] < data['TP ' + ticker][i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
        else:
            positive_flow.append(0)
            negative_flow.append(0)

    #Get all of the positive and negative money flows within the time period
    positive_mf = []
    negative_mf = []
    for i in range(period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1 - period: i+1]))
    for i in range(period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1 - period: i+1]))    

    #Calculate the money flow index
    mfi = 100*(np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf))) 
    df_MFI['MFI ' + ticker] = mfi

first_cross = 0
df_Save_MFI = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])

#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(df_MFI)):
        if df_MFI['MFI ' + ticker][i] < 80 and df_MFI['MFI ' + ticker][i] > 50 and first_cross == 0:
            print(ticker, '=> MFI cắt xuống 80 => Ngày:', df_MFI.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1                
            df_Save_MFI = df_Save_MFI.append(dict(zip(df_Save_MFI.columns,[ticker, 'MFI cross down 80 ', df_MFI.index[i], 'SELL'])), ignore_index=True)
                
        elif df_MFI['MFI ' + ticker][i] > 80 and first_cross == 1:
            print(ticker, '=> MFI cắt lên 80 => Ngày:', df_MFI.index[i].strftime("%d/%m/%Y"), ': Chờ xem SELL')
            first_cross = 0               
            df_Save_MFI = df_Save_MFI.append(dict(zip(df_Save_MFI.columns,[ticker, 'MFI cross up 80 ', df_MFI.index[i], 'Wait to SELL'])), ignore_index=True)
                
        elif df_MFI['MFI ' + ticker][i] > 20 and df_MFI['MFI ' + ticker][i] < 50 and first_cross == 0:
            print(ticker, '=> MFI cắt lên 20 => Ngày:', df_MFI.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 1
            df_Save_MFI = df_Save_MFI.append(dict(zip(df_Save_MFI.columns,[ticker, 'MFI cross up 20 ', df_MFI.index[i], 'BUY'])), ignore_index=True)
                
        elif df_MFI['MFI ' + ticker][i] < 20 and first_cross == 1:
            print(ticker, '=> MFI cắt xuống 20 => Ngày:', df_MFI.index[i].strftime("%d/%m/%Y"), ': Chờ xem BUY')
            first_cross = 0                
            df_Save_MFI = df_Save_MFI.append(dict(zip(df_Save_MFI.columns,[ticker, 'MFI cross down 20 ', df_MFI.index[i], 'Wait to BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_Save_MFI.to_csv("MFI" +".txt", index=False)
print('Hoàn thành lưu MFI.txt')

ATVI => MFI cắt lên 20 => Ngày: 30/04/2021 : Nên BUY
ADBE => MFI cắt lên 80 => Ngày: 23/06/2021 : Chờ xem SELL
ADBE => MFI cắt xuống 80 => Ngày: 01/07/2021 : Nên SELL
ADBE => MFI cắt lên 80 => Ngày: 06/07/2021 : Chờ xem SELL
ADBE => MFI cắt xuống 80 => Ngày: 12/07/2021 : Nên SELL
AMD => MFI cắt lên 80 => Ngày: 02/06/2021 : Chờ xem SELL
AMD => MFI cắt xuống 80 => Ngày: 07/06/2021 : Nên SELL
AMD => MFI cắt lên 80 => Ngày: 06/07/2021 : Chờ xem SELL
AMD => MFI cắt xuống 80 => Ngày: 07/07/2021 : Nên SELL
AMD => MFI cắt lên 80 => Ngày: 02/08/2021 : Chờ xem SELL
AMD => MFI cắt xuống 80 => Ngày: 05/08/2021 : Nên SELL
ALGN => MFI cắt xuống 20 => Ngày: 13/10/2021 : Chờ xem BUY
ALGN => MFI cắt lên 20 => Ngày: 14/10/2021 : Nên BUY
GOOGL => MFI cắt lên 80 => Ngày: 07/07/2021 : Chờ xem SELL
GOOGL => MFI cắt xuống 80 => Ngày: 08/07/2021 : Nên SELL
GOOGL => MFI cắt lên 80 => Ngày: 09/07/2021 : Chờ xem SELL
GOOGL => MFI cắt xuống 80 => Ngày: 12/07/2021 : Nên SELL
GOOG => MFI cắt lên 80 => Ngày: 02/06/2

ISRG => MFI cắt xuống 80 => Ngày: 08/07/2021 : Nên SELL
ISRG => MFI cắt lên 80 => Ngày: 09/07/2021 : Chờ xem SELL
ISRG => MFI cắt xuống 80 => Ngày: 14/07/2021 : Nên SELL
ISRG => MFI cắt lên 80 => Ngày: 05/08/2021 : Chờ xem SELL
ISRG => MFI cắt xuống 80 => Ngày: 11/08/2021 : Nên SELL
JD => MFI cắt xuống 20 => Ngày: 21/06/2021 : Chờ xem BUY
JD => MFI cắt lên 20 => Ngày: 22/06/2021 : Nên BUY
KDP => MFI cắt lên 80 => Ngày: 14/07/2021 : Chờ xem SELL
KDP => MFI cắt xuống 80 => Ngày: 20/07/2021 : Nên SELL
KLAC => MFI cắt xuống 20 => Ngày: 19/07/2021 : Chờ xem BUY
KLAC => MFI cắt lên 20 => Ngày: 20/07/2021 : Nên BUY
KLAC => MFI cắt lên 80 => Ngày: 13/09/2021 : Chờ xem SELL
KLAC => MFI cắt xuống 80 => Ngày: 20/09/2021 : Nên SELL
KLAC => MFI cắt xuống 20 => Ngày: 13/10/2021 : Chờ xem BUY
KLAC => MFI cắt lên 20 => Ngày: 14/10/2021 : Nên BUY
KHC => MFI cắt xuống 20 => Ngày: 24/06/2021 : Chờ xem BUY
KHC => MFI cắt lên 20 => Ngày: 25/06/2021 : Nên BUY
LRCX => MFI cắt xuống 20 => Ngày: 20/08/2021 : C

# Bollinger Bands

In [9]:
first_cross = 0
df_BB = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
    
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data[ticker][i] > data['Upper ' + ticker][i]:
            print(ticker, '=> Giá trên BB => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1                
            df_BB = df_BB.append(dict(zip(df_BB.columns,[ticker, ' Giá trên BB ', data.index[i], 'SELL'])), ignore_index=True)
              
        elif data[ticker][i] < data['Lower ' + ticker][i]:
            print(ticker, '=> Giá dưới BB => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 0                
            df_BB = df_BB.append(dict(zip(df_BB.columns,[ticker, ' Giá dưới BB ', data.index[i], 'BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_BB.to_csv("Bollinger Bands" +".txt", index=False)
print('Hoàn thành lưu Bollinger Bands.txt')

ATVI => Giá dưới BB => Ngày: 04/05/2021 : Nên BUY
ATVI => Giá trên BB => Ngày: 20/05/2021 : Nên SELL
ATVI => Giá dưới BB => Ngày: 17/06/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 18/06/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 21/06/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 27/07/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 28/07/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 29/07/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 03/08/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 07/09/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 09/09/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 20/09/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 21/09/2021 : Nên BUY
ATVI => Giá dưới BB => Ngày: 22/09/2021 : Nên BUY
ADBE => Giá dưới BB => Ngày: 04/05/2021 : Nên BUY
ADBE => Giá dưới BB => Ngày: 05/05/2021 : Nên BUY
ADBE => Giá dưới BB => Ngày: 06/05/2021 : Nên BUY
ADBE => Giá dưới BB => Ngày: 10/05/2021 : Nên BUY
ADBE => Giá trên BB => Ngày: 10/06/2021 : Nên SELL
ADBE => Giá trên BB => Ngày: 11/06/2021 : Nên SE

ADSK => Giá trên BB => Ngày: 26/07/2021 : Nên SELL
ADSK => Giá trên BB => Ngày: 29/07/2021 : Nên SELL
ADSK => Giá trên BB => Ngày: 25/08/2021 : Nên SELL
ADSK => Giá dưới BB => Ngày: 26/08/2021 : Nên BUY
ADSK => Giá dưới BB => Ngày: 01/09/2021 : Nên BUY
ADSK => Giá dưới BB => Ngày: 02/09/2021 : Nên BUY
ADSK => Giá dưới BB => Ngày: 03/09/2021 : Nên BUY
ADSK => Giá dưới BB => Ngày: 04/10/2021 : Nên BUY
ADP => Giá trên BB => Ngày: 10/06/2021 : Nên SELL
ADP => Giá trên BB => Ngày: 07/07/2021 : Nên SELL
ADP => Giá trên BB => Ngày: 03/08/2021 : Nên SELL
ADP => Giá dưới BB => Ngày: 07/09/2021 : Nên BUY
ADP => Giá trên BB => Ngày: 06/10/2021 : Nên SELL
ADP => Giá trên BB => Ngày: 07/10/2021 : Nên SELL
ADP => Giá trên BB => Ngày: 15/10/2021 : Nên SELL
BIDU => Giá dưới BB => Ngày: 04/05/2021 : Nên BUY
BIDU => Giá dưới BB => Ngày: 05/05/2021 : Nên BUY
BIDU => Giá dưới BB => Ngày: 06/05/2021 : Nên BUY
BIDU => Giá dưới BB => Ngày: 07/05/2021 : Nên BUY
BIDU => Giá dưới BB => Ngày: 10/05/2021 : Nên BU

CRWD => Giá trên BB => Ngày: 18/06/2021 : Nên SELL
CRWD => Giá trên BB => Ngày: 22/06/2021 : Nên SELL
CRWD => Giá dưới BB => Ngày: 11/08/2021 : Nên BUY
CRWD => Giá dưới BB => Ngày: 16/08/2021 : Nên BUY
CRWD => Giá dưới BB => Ngày: 17/08/2021 : Nên BUY
CRWD => Giá dưới BB => Ngày: 28/09/2021 : Nên BUY
CRWD => Giá dưới BB => Ngày: 29/09/2021 : Nên BUY
CRWD => Giá trên BB => Ngày: 14/10/2021 : Nên SELL
CSX => Giá trên BB => Ngày: 21/04/2021 : Nên SELL
CSX => Giá trên BB => Ngày: 22/04/2021 : Nên SELL
CSX => Giá trên BB => Ngày: 23/04/2021 : Nên SELL
CSX => Giá dưới BB => Ngày: 18/05/2021 : Nên BUY
CSX => Giá dưới BB => Ngày: 19/05/2021 : Nên BUY
CSX => Giá dưới BB => Ngày: 10/06/2021 : Nên BUY
CSX => Giá dưới BB => Ngày: 18/06/2021 : Nên BUY
CSX => Giá dưới BB => Ngày: 08/07/2021 : Nên BUY
CSX => Giá dưới BB => Ngày: 19/07/2021 : Nên BUY
CSX => Giá trên BB => Ngày: 11/08/2021 : Nên SELL
CSX => Giá trên BB => Ngày: 12/08/2021 : Nên SELL
CSX => Giá dưới BB => Ngày: 03/09/2021 : Nên BUY
CSX 

ILMN => Giá dưới BB => Ngày: 19/08/2021 : Nên BUY
ILMN => Giá dưới BB => Ngày: 28/09/2021 : Nên BUY
ILMN => Giá dưới BB => Ngày: 01/10/2021 : Nên BUY
ILMN => Giá dưới BB => Ngày: 04/10/2021 : Nên BUY
INCY => Giá trên BB => Ngày: 07/06/2021 : Nên SELL
INCY => Giá trên BB => Ngày: 09/06/2021 : Nên SELL
INCY => Giá trên BB => Ngày: 10/06/2021 : Nên SELL
INCY => Giá dưới BB => Ngày: 16/07/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 09/08/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 10/08/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 11/08/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 12/08/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 10/09/2021 : Nên BUY
INCY => Giá dưới BB => Ngày: 22/09/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 23/04/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 26/04/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 27/04/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 18/06/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 23/06/2021 : Nên BUY
INTC => Giá dưới BB => Ngày: 19/07/2021 : Nên B

MRNA => Giá trên BB => Ngày: 02/06/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 03/06/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 04/06/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 07/06/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 29/06/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 15/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 16/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 19/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 20/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 21/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 23/07/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 09/08/2021 : Nên SELL
MRNA => Giá trên BB => Ngày: 09/09/2021 : Nên SELL
MRNA => Giá dưới BB => Ngày: 29/09/2021 : Nên BUY
MRNA => Giá dưới BB => Ngày: 01/10/2021 : Nên BUY
MRNA => Giá dưới BB => Ngày: 04/10/2021 : Nên BUY
MRNA => Giá dưới BB => Ngày: 06/10/2021 : Nên BUY
MDLZ => Giá trên BB => Ngày: 28/04/2021 : Nên SELL
MDLZ => Giá trên BB => Ngày: 29/04/2021 : Nên SELL
MDLZ => Giá trên BB => Ngày: 30/04/

QCOM => Giá trên BB => Ngày: 29/07/2021 : Nên SELL
QCOM => Giá trên BB => Ngày: 30/07/2021 : Nên SELL
QCOM => Giá dưới BB => Ngày: 15/09/2021 : Nên BUY
QCOM => Giá dưới BB => Ngày: 17/09/2021 : Nên BUY
QCOM => Giá dưới BB => Ngày: 20/09/2021 : Nên BUY
QCOM => Giá dưới BB => Ngày: 21/09/2021 : Nên BUY
REGN => Giá trên BB => Ngày: 15/04/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 16/04/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 20/04/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 13/05/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 17/05/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 10/06/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 01/07/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 02/07/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 06/07/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 05/08/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 06/08/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 09/08/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 10/08/2021 : Nên SELL
REGN => Giá trên BB => Ngày: 13/08/

WBA => Giá dưới BB => Ngày: 02/07/2021 : Nên BUY
WBA => Giá dưới BB => Ngày: 06/07/2021 : Nên BUY
WBA => Giá trên BB => Ngày: 10/08/2021 : Nên SELL
WBA => Giá trên BB => Ngày: 11/08/2021 : Nên SELL
WBA => Giá trên BB => Ngày: 12/08/2021 : Nên SELL
WBA => Giá trên BB => Ngày: 31/08/2021 : Nên SELL
WBA => Giá trên BB => Ngày: 02/09/2021 : Nên SELL
WBA => Giá trên BB => Ngày: 14/10/2021 : Nên SELL
WDAY => Giá trên BB => Ngày: 13/04/2021 : Nên SELL
WDAY => Giá dưới BB => Ngày: 30/04/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 03/05/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 04/05/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 05/05/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 06/05/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 12/05/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 15/07/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 16/07/2021 : Nên BUY
WDAY => Giá dưới BB => Ngày: 19/07/2021 : Nên BUY
WDAY => Giá trên BB => Ngày: 23/08/2021 : Nên SELL
WDAY => Giá trên BB => Ngày: 24/08/2021 : Nên SELL

# Stochastic

In [10]:
df_Stochastic = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data['%K ' + ticker][i] < data['%D ' + ticker][i] and data['%D ' + ticker][i] > 80:
            print(ticker, '=> %K cắt xuống %D => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')               
            df_Stochastic = df_Stochastic.append(dict(zip(df_Stochastic.columns,[ticker, ' %K cắt xuống %D ', data.index[i], 'SELL'])), ignore_index=True)
                
        elif data['%K ' + ticker][i] > data['%D ' + ticker][i] and data['%D ' + ticker][i] < 20:
            print(ticker, '=> %K cắt lên %D => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')               
            df_Stochastic = df_Stochastic.append(dict(zip(df_Stochastic.columns,[ticker, ' %K cắt lên %D ', data.index[i], 'BUY'])), ignore_index=True)
            
#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_Stochastic.to_csv("Stochastic" +".txt", index=False)
print('Hoàn thành lưu Stochastic.txt')

ATVI => %K cắt xuống %D => Ngày: 16/04/2021 : Nên SELL
ATVI => %K cắt xuống %D => Ngày: 19/04/2021 : Nên SELL
ATVI => %K cắt lên %D => Ngày: 29/04/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 04/05/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 05/05/2021 : Nên BUY
ATVI => %K cắt xuống %D => Ngày: 21/05/2021 : Nên SELL
ATVI => %K cắt xuống %D => Ngày: 26/05/2021 : Nên SELL
ATVI => %K cắt xuống %D => Ngày: 27/05/2021 : Nên SELL
ATVI => %K cắt xuống %D => Ngày: 28/05/2021 : Nên SELL
ATVI => %K cắt lên %D => Ngày: 21/06/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 22/06/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 24/06/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 25/06/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 16/07/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 20/07/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 28/07/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 03/08/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 04/08/2021 : Nên BUY
ATVI => %K cắt lên %D => Ngày: 15/09/2021 : Nê

GOOG => %K cắt xuống %D => Ngày: 11/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 15/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 16/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 18/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 23/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 25/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 28/06/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 07/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 08/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 09/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 13/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 14/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 15/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 23/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 27/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 28/07/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 26/08/2021 : Nên SELL
GOOG => %K cắt xuống %D => Ngày: 31/08/2021 : Nên SELL
GOOG => %K

ANSS => %K cắt xuống %D => Ngày: 09/09/2021 : Nên SELL
ANSS => %K cắt xuống %D => Ngày: 10/09/2021 : Nên SELL
ANSS => %K cắt lên %D => Ngày: 22/09/2021 : Nên BUY
ANSS => %K cắt lên %D => Ngày: 01/10/2021 : Nên BUY
ANSS => %K cắt lên %D => Ngày: 05/10/2021 : Nên BUY
AAPL => %K cắt xuống %D => Ngày: 12/04/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 14/04/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 20/04/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 21/04/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 27/04/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 28/04/2021 : Nên SELL
AAPL => %K cắt lên %D => Ngày: 06/05/2021 : Nên BUY
AAPL => %K cắt lên %D => Ngày: 13/05/2021 : Nên BUY
AAPL => %K cắt lên %D => Ngày: 14/05/2021 : Nên BUY
AAPL => %K cắt xuống %D => Ngày: 15/06/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 16/06/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 18/06/2021 : Nên SELL
AAPL => %K cắt xuống %D => Ngày: 23/06/2021 : Nên SELL
AAPL => %K cắt xuống %D => N

ADSK => %K cắt xuống %D => Ngày: 28/09/2021 : Nên SELL
ADSK => %K cắt lên %D => Ngày: 06/10/2021 : Nên BUY
ADP => %K cắt xuống %D => Ngày: 20/04/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 22/04/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 23/04/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 26/04/2021 : Nên SELL
ADP => %K cắt lên %D => Ngày: 30/04/2021 : Nên BUY
ADP => %K cắt xuống %D => Ngày: 10/05/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 25/05/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 03/06/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 07/06/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 09/06/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 14/06/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 15/06/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 06/07/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 08/07/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 13/07/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 14/07/2021 : Nên SELL
ADP => %K cắt xuống %D => Ngày: 

CERN => %K cắt xuống %D => Ngày: 23/04/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 26/04/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 27/04/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 21/05/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 14/06/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 15/06/2021 : Nên SELL
CERN => %K cắt lên %D => Ngày: 25/06/2021 : Nên BUY
CERN => %K cắt xuống %D => Ngày: 08/07/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 09/07/2021 : Nên SELL
CERN => %K cắt xuống %D => Ngày: 04/08/2021 : Nên SELL
CERN => %K cắt lên %D => Ngày: 25/08/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 27/08/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 02/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 10/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 13/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 15/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 16/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 23/09/2021 : Nên BUY
CERN => %K cắt lên %D => Ngày: 24/09/

...

MRNA => %K cắt xuống %D => Ngày: 27/07/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 02/08/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 04/08/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 05/08/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 06/08/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 10/08/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 08/09/2021 : Nên SELL
MRNA => %K cắt xuống %D => Ngày: 10/09/2021 : Nên SELL
MRNA => %K cắt lên %D => Ngày: 30/09/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 01/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 04/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 05/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 07/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 08/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 12/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 13/10/2021 : Nên BUY
MRNA => %K cắt lên %D => Ngày: 14/10/2021 : Nên BUY
MDLZ => %K cắt xuống %D => Ngày: 23/04/2021 : Nên SELL
MDLZ => %K cắt xuống %D => Ngày: 07/0

ORLY => %K cắt xuống %D => Ngày: 27/09/2021 : Nên SELL
ORLY => %K cắt xuống %D => Ngày: 28/09/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 14/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 16/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 19/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 20/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 22/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 23/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 28/04/2021 : Nên SELL
OKTA => %K cắt xuống %D => Ngày: 29/04/2021 : Nên SELL
OKTA => %K cắt lên %D => Ngày: 05/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 07/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 11/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 14/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 18/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 19/05/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 04/06/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 07/06/2021 : Nên BUY
OKTA => %K cắt lên %D => Ngày: 08/

PDD => %K cắt xuống %D => Ngày: 10/09/2021 : Nên SELL
PDD => %K cắt lên %D => Ngày: 30/09/2021 : Nên BUY
PDD => %K cắt lên %D => Ngày: 05/10/2021 : Nên BUY
PDD => %K cắt lên %D => Ngày: 06/10/2021 : Nên BUY
QCOM => %K cắt xuống %D => Ngày: 12/04/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 13/04/2021 : Nên SELL
QCOM => %K cắt lên %D => Ngày: 06/05/2021 : Nên BUY
QCOM => %K cắt lên %D => Ngày: 13/05/2021 : Nên BUY
QCOM => %K cắt lên %D => Ngày: 14/05/2021 : Nên BUY
QCOM => %K cắt xuống %D => Ngày: 01/06/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 02/06/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 15/06/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 30/06/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 01/07/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 06/07/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 13/07/2021 : Nên SELL
QCOM => %K cắt xuống %D => Ngày: 02/08/2021 : Nên SELL
QCOM => %K cắt lên %D => Ngày: 19/08/2021 : Nên BUY
QCOM => %K cắt lên %D => Ngày: 20/0

SBUX => %K cắt xuống %D => Ngày: 19/04/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 20/04/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 21/04/2021 : Nên SELL
SBUX => %K cắt lên %D => Ngày: 13/05/2021 : Nên BUY
SBUX => %K cắt lên %D => Ngày: 14/05/2021 : Nên BUY
SBUX => %K cắt xuống %D => Ngày: 02/06/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 29/06/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 07/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 08/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 13/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 14/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 15/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 16/07/2021 : Nên SELL
SBUX => %K cắt xuống %D => Ngày: 28/07/2021 : Nên SELL
SBUX => %K cắt lên %D => Ngày: 11/08/2021 : Nên BUY
SBUX => %K cắt lên %D => Ngày: 13/08/2021 : Nên BUY
SBUX => %K cắt lên %D => Ngày: 16/08/2021 : Nên BUY
SBUX => %K cắt lên %D => Ngày: 20/08/2021 : Nên BUY
SBUX => %K cắt lên %D => Ngà

VRSK => %K cắt lên %D => Ngày: 07/05/2021 : Nên BUY
VRSK => %K cắt lên %D => Ngày: 13/05/2021 : Nên BUY
VRSK => %K cắt lên %D => Ngày: 14/05/2021 : Nên BUY
VRSK => %K cắt lên %D => Ngày: 19/05/2021 : Nên BUY
VRSK => %K cắt lên %D => Ngày: 20/05/2021 : Nên BUY
VRSK => %K cắt xuống %D => Ngày: 06/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 07/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 08/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 09/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 16/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 19/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 22/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 26/07/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 02/08/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 03/08/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 16/08/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 18/08/2021 : Nên SELL
VRSK => %K cắt xuống %D => Ngày: 24/08/2021 : Nên SELL
VRSK => %K cắt xuống %D =

# Parabolic SAR

In [11]:
first_cross = 0
df_SAR = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data['SAR ' + ticker][i] < data[ticker][i] and first_cross == 0:
            print(ticker, '=> SAR dưới Close => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 1                
            df_SAR = df_SAR.append(dict(zip(df_SAR.columns,[ticker, ' SAR dưới Close ', data.index[i], 'BUY'])), ignore_index=True)
                
        elif data['SAR ' + ticker][i] > data[ticker][i] and first_cross == 1:
            print(ticker, '=> SAR trên Close => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 0               
            df_SAR = df_SAR.append(dict(zip(df_SAR.columns,[ticker, ' SAR trên Close ', data.index[i], 'SELL'])), ignore_index=True)

#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_SAR.to_csv("Parabolic SAR" +".txt", index=False)            
print('Hoàn thành lưu Parabolic SAR.txt')

ATVI => SAR dưới Close => Ngày: 12/04/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 21/04/2021 : Nên SELL
ATVI => SAR dưới Close => Ngày: 05/05/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 02/06/2021 : Nên SELL
ATVI => SAR dưới Close => Ngày: 09/06/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 16/06/2021 : Nên SELL
ATVI => SAR dưới Close => Ngày: 28/06/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 15/07/2021 : Nên SELL
ATVI => SAR dưới Close => Ngày: 11/08/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 25/08/2021 : Nên SELL
ATVI => SAR dưới Close => Ngày: 28/09/2021 : Nên BUY
ATVI => SAR trên Close => Ngày: 12/10/2021 : Nên SELL
ADBE => SAR dưới Close => Ngày: 12/04/2021 : Nên BUY
ADBE => SAR trên Close => Ngày: 20/04/2021 : Nên SELL
ADBE => SAR dưới Close => Ngày: 29/04/2021 : Nên BUY
ADBE => SAR trên Close => Ngày: 03/05/2021 : Nên SELL
ADBE => SAR dưới Close => Ngày: 20/05/2021 : Nên BUY
ADBE => SAR trên Close => Ngày: 03/06/2021 : Nên SELL
ADBE => SAR dưới Close => Ngày: 08/06

ASML => SAR dưới Close => Ngày: 21/07/2021 : Nên BUY
ASML => SAR trên Close => Ngày: 11/08/2021 : Nên SELL
ASML => SAR dưới Close => Ngày: 23/08/2021 : Nên BUY
ASML => SAR trên Close => Ngày: 17/09/2021 : Nên SELL
ASML => SAR dưới Close => Ngày: 23/09/2021 : Nên BUY
ASML => SAR trên Close => Ngày: 28/09/2021 : Nên SELL
ASML => SAR dưới Close => Ngày: 14/10/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 29/04/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 20/05/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 03/06/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 10/06/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 29/06/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 06/07/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 13/07/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 23/07/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 13/09/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 14/09/2021 : Nên BUY
TEAM => SAR trên Close => Ngày: 27/09/2021 : Nên SELL
TEAM => SAR dưới Close => Ngày: 13/10

CHKP => SAR dưới Close => Ngày: 29/07/2021 : Nên BUY
CHKP => SAR trên Close => Ngày: 12/08/2021 : Nên SELL
CHKP => SAR dưới Close => Ngày: 24/08/2021 : Nên BUY
CHKP => SAR trên Close => Ngày: 07/09/2021 : Nên SELL
CHKP => SAR dưới Close => Ngày: 07/10/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 20/04/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 05/05/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 12/05/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 01/06/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 13/07/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 20/07/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 11/08/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 16/08/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 07/09/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 09/09/2021 : Nên BUY
CTAS => SAR trên Close => Ngày: 17/09/2021 : Nên SELL
CTAS => SAR dưới Close => Ngày: 06/10/2021 : Nên BUY
CSCO => SAR trên Close => Ngày: 12/04/2021 : Nên SELL
CSCO => SAR dưới Close => Ngày: 16/04

EXC => SAR dưới Close => Ngày: 05/10/2021 : Nên BUY
EXC => SAR trên Close => Ngày: 11/10/2021 : Nên SELL
EXC => SAR dưới Close => Ngày: 14/10/2021 : Nên BUY
FB => SAR trên Close => Ngày: 14/04/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 28/04/2021 : Nên BUY
FB => SAR trên Close => Ngày: 10/05/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 21/05/2021 : Nên BUY
FB => SAR trên Close => Ngày: 16/06/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 22/06/2021 : Nên BUY
FB => SAR trên Close => Ngày: 08/07/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 13/07/2021 : Nên BUY
FB => SAR trên Close => Ngày: 15/07/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 23/07/2021 : Nên BUY
FB => SAR trên Close => Ngày: 19/08/2021 : Nên SELL
FB => SAR dưới Close => Ngày: 30/08/2021 : Nên BUY
FB => SAR trên Close => Ngày: 17/09/2021 : Nên SELL
FAST => SAR dưới Close => Ngày: 19/04/2021 : Nên BUY
FAST => SAR trên Close => Ngày: 12/05/2021 : Nên SELL
FAST => SAR dưới Close => Ngày: 27/05/2021 : Nên BUY
FAST => SAR t

ISRG => SAR trên Close => Ngày: 29/04/2021 : Nên SELL
ISRG => SAR dưới Close => Ngày: 20/05/2021 : Nên BUY
ISRG => SAR trên Close => Ngày: 02/06/2021 : Nên SELL
ISRG => SAR dưới Close => Ngày: 10/06/2021 : Nên BUY
ISRG => SAR trên Close => Ngày: 15/07/2021 : Nên SELL
ISRG => SAR dưới Close => Ngày: 20/07/2021 : Nên BUY
ISRG => SAR trên Close => Ngày: 11/08/2021 : Nên SELL
ISRG => SAR dưới Close => Ngày: 17/08/2021 : Nên BUY
ISRG => SAR trên Close => Ngày: 10/09/2021 : Nên SELL
ISRG => SAR dưới Close => Ngày: 07/10/2021 : Nên BUY
ISRG => SAR trên Close => Ngày: 12/10/2021 : Nên SELL
JD => SAR dưới Close => Ngày: 27/04/2021 : Nên BUY
JD => SAR trên Close => Ngày: 06/05/2021 : Nên SELL
JD => SAR dưới Close => Ngày: 20/05/2021 : Nên BUY
JD => SAR trên Close => Ngày: 10/06/2021 : Nên SELL
JD => SAR dưới Close => Ngày: 25/06/2021 : Nên BUY
JD => SAR trên Close => Ngày: 06/07/2021 : Nên SELL
JD => SAR dưới Close => Ngày: 15/07/2021 : Nên BUY
JD => SAR trên Close => Ngày: 23/07/2021 : Nên SELL

MDLZ => SAR dưới Close => Ngày: 23/07/2021 : Nên BUY
MDLZ => SAR trên Close => Ngày: 28/07/2021 : Nên SELL
MDLZ => SAR dưới Close => Ngày: 16/08/2021 : Nên BUY
MDLZ => SAR trên Close => Ngày: 25/08/2021 : Nên SELL
MDLZ => SAR dưới Close => Ngày: 05/10/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 26/04/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 03/05/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 04/05/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 21/05/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 09/06/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 25/06/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 08/07/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 20/07/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 03/08/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 06/08/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 03/09/2021 : Nên SELL
MNST => SAR dưới Close => Ngày: 07/10/2021 : Nên BUY
MNST => SAR trên Close => Ngày: 11/10/2021 : Nên SELL
NTES => SAR dưới Close => Ngày: 12/04

QCOM => SAR dưới Close => Ngày: 24/05/2021 : Nên BUY
QCOM => SAR trên Close => Ngày: 16/06/2021 : Nên SELL
QCOM => SAR dưới Close => Ngày: 24/06/2021 : Nên BUY
QCOM => SAR trên Close => Ngày: 08/07/2021 : Nên SELL
QCOM => SAR dưới Close => Ngày: 14/07/2021 : Nên BUY
QCOM => SAR trên Close => Ngày: 10/08/2021 : Nên SELL
QCOM => SAR dưới Close => Ngày: 30/08/2021 : Nên BUY
QCOM => SAR trên Close => Ngày: 14/09/2021 : Nên SELL
QCOM => SAR dưới Close => Ngày: 07/10/2021 : Nên BUY
QCOM => SAR trên Close => Ngày: 11/10/2021 : Nên SELL
QCOM => SAR dưới Close => Ngày: 15/10/2021 : Nên BUY
REGN => SAR trên Close => Ngày: 23/04/2021 : Nên SELL
REGN => SAR dưới Close => Ngày: 06/05/2021 : Nên BUY
REGN => SAR trên Close => Ngày: 24/05/2021 : Nên SELL
REGN => SAR dưới Close => Ngày: 04/06/2021 : Nên BUY
REGN => SAR trên Close => Ngày: 13/07/2021 : Nên SELL
REGN => SAR dưới Close => Ngày: 15/07/2021 : Nên BUY
REGN => SAR trên Close => Ngày: 27/07/2021 : Nên SELL
REGN => SAR dưới Close => Ngày: 04/08

VRSK => SAR dưới Close => Ngày: 08/10/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 12/04/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 13/04/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 22/04/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 30/04/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 19/05/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 04/06/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 11/06/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 28/06/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 14/07/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 16/07/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 20/07/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 28/07/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 10/08/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 23/08/2021 : Nên BUY
VRTX => SAR trên Close => Ngày: 07/09/2021 : Nên SELL
VRTX => SAR dưới Close => Ngày: 06/10/2021 : Nên BUY
WBA => SAR trên Close => Ngày: 19/04/2021 : Nên SELL
WBA => SAR dưới Close => Ngày: 03/05/2

# ADX

In [12]:
first_cross = 0
df_ADX = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data['ADX ' + ticker][i] < 25 and first_cross == 0:
            print(ticker, '=> ADX cắt xuống 25 => Ngày:', data.index[i].strftime("%d/%m/%Y"), 'No Trend')
            first_cross = 1
            df_ADX = df_ADX.append(dict(zip(df_ADX.columns,[ticker, ' ADX cắt xuống 25 ', data.index[i], 'No Trend'])), ignore_index=True)
                
        elif data['ADX ' + ticker][i] > 25 and first_cross == 1:
            print(ticker, '=> ADX cắt lên 25 => Ngày:', data.index[i].strftime("%d/%m/%Y"), 'Trend')
            first_cross = 0               
            df_ADX = df_ADX.append(dict(zip(df_ADX.columns,[ticker, ' ADX cắt lên 25 ', data.index[i], 'Trend'])), ignore_index=True)

#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_ADX.to_csv("ADX" +".txt", index=False)            
print('Hoàn thành lưu ADX.txt')

ATVI => ADX cắt xuống 25 => Ngày: 12/04/2021 No Trend
ATVI => ADX cắt lên 25 => Ngày: 27/07/2021 Trend
ATVI => ADX cắt xuống 25 => Ngày: 05/10/2021 No Trend
ADBE => ADX cắt lên 25 => Ngày: 13/04/2021 Trend
ADBE => ADX cắt xuống 25 => Ngày: 05/05/2021 No Trend
ADBE => ADX cắt lên 25 => Ngày: 06/05/2021 Trend
ADBE => ADX cắt xuống 25 => Ngày: 07/05/2021 No Trend
ADBE => ADX cắt lên 25 => Ngày: 21/06/2021 Trend
AMD => ADX cắt xuống 25 => Ngày: 12/04/2021 No Trend
AMD => ADX cắt lên 25 => Ngày: 30/06/2021 Trend
AMD => ADX cắt xuống 25 => Ngày: 19/07/2021 No Trend
AMD => ADX cắt lên 25 => Ngày: 02/08/2021 Trend
AMD => ADX cắt xuống 25 => Ngày: 10/09/2021 No Trend
ALGN => ADX cắt lên 25 => Ngày: 22/04/2021 Trend
ALGN => ADX cắt xuống 25 => Ngày: 10/05/2021 No Trend
ALGN => ADX cắt lên 25 => Ngày: 13/10/2021 Trend
GOOGL => ADX cắt xuống 25 => Ngày: 12/04/2021 No Trend
GOOGL => ADX cắt lên 25 => Ngày: 13/04/2021 Trend
GOOGL => ADX cắt xuống 25 => Ngày: 18/05/2021 No Trend
GOOGL => ADX cắt lên 

DOCU => ADX cắt xuống 25 => Ngày: 26/05/2021 No Trend
DOCU => ADX cắt lên 25 => Ngày: 09/06/2021 Trend
DOCU => ADX cắt xuống 25 => Ngày: 16/08/2021 No Trend
DOCU => ADX cắt lên 25 => Ngày: 04/10/2021 Trend
DLTR => ADX cắt xuống 25 => Ngày: 27/04/2021 No Trend
DLTR => ADX cắt lên 25 => Ngày: 21/05/2021 Trend
DLTR => ADX cắt xuống 25 => Ngày: 30/07/2021 No Trend
DLTR => ADX cắt lên 25 => Ngày: 30/08/2021 Trend
EBAY => ADX cắt xuống 25 => Ngày: 12/04/2021 No Trend
EBAY => ADX cắt lên 25 => Ngày: 14/07/2021 Trend
EBAY => ADX cắt xuống 25 => Ngày: 19/07/2021 No Trend
EBAY => ADX cắt lên 25 => Ngày: 22/07/2021 Trend
EBAY => ADX cắt xuống 25 => Ngày: 11/08/2021 No Trend
EBAY => ADX cắt lên 25 => Ngày: 17/08/2021 Trend
EBAY => ADX cắt xuống 25 => Ngày: 24/08/2021 No Trend
EBAY => ADX cắt lên 25 => Ngày: 26/08/2021 Trend
EBAY => ADX cắt xuống 25 => Ngày: 10/09/2021 No Trend
EA => ADX cắt lên 25 => Ngày: 12/04/2021 Trend
EA => ADX cắt xuống 25 => Ngày: 20/04/2021 No Trend
EA => ADX cắt lên 25 =>

ORLY => ADX cắt lên 25 => Ngày: 12/04/2021 Trend
ORLY => ADX cắt xuống 25 => Ngày: 03/06/2021 No Trend
ORLY => ADX cắt lên 25 => Ngày: 29/06/2021 Trend
ORLY => ADX cắt xuống 25 => Ngày: 17/08/2021 No Trend
OKTA => ADX cắt lên 25 => Ngày: 20/04/2021 Trend
OKTA => ADX cắt xuống 25 => Ngày: 18/06/2021 No Trend
OKTA => ADX cắt lên 25 => Ngày: 02/09/2021 Trend
OKTA => ADX cắt xuống 25 => Ngày: 13/09/2021 No Trend
OKTA => ADX cắt lên 25 => Ngày: 11/10/2021 Trend
OKTA => ADX cắt xuống 25 => Ngày: 13/10/2021 No Trend
PCAR => ADX cắt lên 25 => Ngày: 06/07/2021 Trend
PCAR => ADX cắt xuống 25 => Ngày: 15/07/2021 No Trend
PCAR => ADX cắt lên 25 => Ngày: 04/08/2021 Trend
PCAR => ADX cắt xuống 25 => Ngày: 30/08/2021 No Trend
PAYX => ADX cắt lên 25 => Ngày: 09/07/2021 Trend
PAYX => ADX cắt xuống 25 => Ngày: 31/08/2021 No Trend
PAYX => ADX cắt lên 25 => Ngày: 07/09/2021 Trend
PYPL => ADX cắt xuống 25 => Ngày: 12/04/2021 No Trend
PYPL => ADX cắt lên 25 => Ngày: 19/05/2021 Trend
PYPL => ADX cắt xuống 25

# SMA20-50

In [13]:
#Create a function to see the dates of each death cross and golden cross within the data set
first_cross = 0
df_SMA20_SMA50 = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])
#Loop throught the lenght of the data set
for ticker in tickers_list:
    for i in range(0, len(data)):
        if data['SMA20 ' + ticker][i] < data['SMA50 ' + ticker][i] and first_cross == 0:
            print(ticker, '=> SMA20 cắt xuống SMA50 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên SELL')
            first_cross = 1                
            df_SMA20_SMA50 = df_SMA20_SMA50.append(dict(zip(df_SMA20_SMA50.columns,[ticker, ' SMA20 cắt xuống SMA50 ', data.index[i], 'SELL'])), ignore_index=True)
                
        elif data['SMA20 ' + ticker][i] > data['SMA50 ' + ticker][i] and first_cross == 1:
            print(ticker, '=> SMA20 cắt lên SMA50 => Ngày:', data.index[i].strftime("%d/%m/%Y"), ': Nên BUY')
            first_cross = 0                
            df_SMA20_SMA50 = df_SMA20_SMA50.append(dict(zip(df_SMA20_SMA50.columns,[ticker, ' SMA20 cắt lên SMA50 ', data.index[i], 'BUY'])), ignore_index=True)

#os.chdir(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic/{str_today_weekday}')
#df_SMA20_SMA50.to_csv("SMA20 & SMA50" +".txt", index=False)            
print('Hoàn thành lưu SMA20 & SMA50.txt')

ATVI => SMA20 cắt xuống SMA50 => Ngày: 12/04/2021 : Nên SELL
ATVI => SMA20 cắt lên SMA50 => Ngày: 23/04/2021 : Nên BUY
ATVI => SMA20 cắt xuống SMA50 => Ngày: 13/05/2021 : Nên SELL
ATVI => SMA20 cắt lên SMA50 => Ngày: 27/05/2021 : Nên BUY
ATVI => SMA20 cắt xuống SMA50 => Ngày: 08/07/2021 : Nên SELL
ADBE => SMA20 cắt lên SMA50 => Ngày: 13/04/2021 : Nên BUY
ADBE => SMA20 cắt xuống SMA50 => Ngày: 27/05/2021 : Nên SELL
ADBE => SMA20 cắt lên SMA50 => Ngày: 14/06/2021 : Nên BUY
ADBE => SMA20 cắt xuống SMA50 => Ngày: 01/10/2021 : Nên SELL
AMD => SMA20 cắt lên SMA50 => Ngày: 28/04/2021 : Nên BUY
AMD => SMA20 cắt xuống SMA50 => Ngày: 14/05/2021 : Nên SELL
AMD => SMA20 cắt lên SMA50 => Ngày: 16/06/2021 : Nên BUY
AMD => SMA20 cắt xuống SMA50 => Ngày: 30/09/2021 : Nên SELL
ALGN => SMA20 cắt lên SMA50 => Ngày: 20/04/2021 : Nên BUY
ALGN => SMA20 cắt xuống SMA50 => Ngày: 04/06/2021 : Nên SELL
ALGN => SMA20 cắt lên SMA50 => Ngày: 15/06/2021 : Nên BUY
ALGN => SMA20 cắt xuống SMA50 => Ngày: 06/10/2021 : 

DOCU => SMA20 cắt lên SMA50 => Ngày: 14/06/2021 : Nên BUY
DOCU => SMA20 cắt xuống SMA50 => Ngày: 08/09/2021 : Nên SELL
DLTR => SMA20 cắt lên SMA50 => Ngày: 12/04/2021 : Nên BUY
DLTR => SMA20 cắt xuống SMA50 => Ngày: 19/05/2021 : Nên SELL
DLTR => SMA20 cắt lên SMA50 => Ngày: 16/08/2021 : Nên BUY
DLTR => SMA20 cắt xuống SMA50 => Ngày: 01/09/2021 : Nên SELL
EBAY => SMA20 cắt lên SMA50 => Ngày: 12/04/2021 : Nên BUY
EBAY => SMA20 cắt xuống SMA50 => Ngày: 17/05/2021 : Nên SELL
EBAY => SMA20 cắt lên SMA50 => Ngày: 07/06/2021 : Nên BUY
EBAY => SMA20 cắt xuống SMA50 => Ngày: 06/10/2021 : Nên SELL
EBAY => SMA20 cắt lên SMA50 => Ngày: 07/10/2021 : Nên BUY
EBAY => SMA20 cắt xuống SMA50 => Ngày: 14/10/2021 : Nên SELL
EA => SMA20 cắt lên SMA50 => Ngày: 19/04/2021 : Nên BUY
EA => SMA20 cắt xuống SMA50 => Ngày: 13/07/2021 : Nên SELL
EA => SMA20 cắt lên SMA50 => Ngày: 23/07/2021 : Nên BUY
EA => SMA20 cắt xuống SMA50 => Ngày: 28/07/2021 : Nên SELL
EA => SMA20 cắt lên SMA50 => Ngày: 07/09/2021 : Nên BUY


PAYX => SMA20 cắt xuống SMA50 => Ngày: 15/09/2021 : Nên SELL
PAYX => SMA20 cắt lên SMA50 => Ngày: 15/10/2021 : Nên BUY
PYPL => SMA20 cắt xuống SMA50 => Ngày: 12/04/2021 : Nên SELL
PYPL => SMA20 cắt lên SMA50 => Ngày: 23/04/2021 : Nên BUY
PYPL => SMA20 cắt xuống SMA50 => Ngày: 21/05/2021 : Nên SELL
PYPL => SMA20 cắt lên SMA50 => Ngày: 15/06/2021 : Nên BUY
PYPL => SMA20 cắt xuống SMA50 => Ngày: 16/08/2021 : Nên SELL
PTON => SMA20 cắt lên SMA50 => Ngày: 11/06/2021 : Nên BUY
PTON => SMA20 cắt xuống SMA50 => Ngày: 18/08/2021 : Nên SELL
PEP => SMA20 cắt lên SMA50 => Ngày: 12/04/2021 : Nên BUY
PEP => SMA20 cắt xuống SMA50 => Ngày: 28/09/2021 : Nên SELL
PDD => SMA20 cắt lên SMA50 => Ngày: 15/09/2021 : Nên BUY
PDD => SMA20 cắt xuống SMA50 => Ngày: 14/10/2021 : Nên SELL
QCOM => SMA20 cắt lên SMA50 => Ngày: 22/04/2021 : Nên BUY
QCOM => SMA20 cắt xuống SMA50 => Ngày: 18/05/2021 : Nên SELL
QCOM => SMA20 cắt lên SMA50 => Ngày: 21/06/2021 : Nên BUY
QCOM => SMA20 cắt xuống SMA50 => Ngày: 14/09/2021 : 

# Đọc file

In [14]:
#Đọc file txt
df_CCI=df_CCI.set_index(pd.DatetimeIndex(df_CCI['Date'].values))  #set index để lọc
df_CCI['Date'] = df_CCI['Date'].dt.strftime('%Y-%m-%d') #chuyển date sang string để to_numpy -> save vô excel

df_RSI=df_RSI.set_index(pd.DatetimeIndex(df_RSI['Date'].values))
df_RSI['Date'] = df_RSI['Date'].dt.strftime('%Y-%m-%d')

df_MACD=df_MACD.set_index(pd.DatetimeIndex(df_MACD['Date'].values))
df_MACD['Date'] = df_MACD['Date'].dt.strftime('%Y-%m-%d')

df_EMA12_EMA36=df_EMA12_EMA36.set_index(pd.DatetimeIndex(df_EMA12_EMA36['Date'].values))
df_EMA12_EMA36['Date'] = df_EMA12_EMA36['Date'].dt.strftime('%Y-%m-%d')

df_Save_MFI=df_Save_MFI.set_index(pd.DatetimeIndex(df_Save_MFI['Date'].values))
df_Save_MFI['Date'] = df_Save_MFI['Date'].dt.strftime('%Y-%m-%d')

df_BB=df_BB.set_index(pd.DatetimeIndex(df_BB['Date'].values))
df_BB['Date'] = df_BB['Date'].dt.strftime('%Y-%m-%d')

df_Stochastic=df_Stochastic.set_index(pd.DatetimeIndex(df_Stochastic['Date'].values))
df_Stochastic['Date'] = df_Stochastic['Date'].dt.strftime('%Y-%m-%d')

df_SAR=df_SAR.set_index(pd.DatetimeIndex(df_SAR['Date'].values))
df_SAR['Date'] = df_SAR['Date'].dt.strftime('%Y-%m-%d') 

df_ADX=df_ADX.set_index(pd.DatetimeIndex(df_ADX['Date'].values)) 
df_ADX['Date'] = df_ADX['Date'].dt.strftime('%Y-%m-%d')

df_SMA20_SMA50=df_SMA20_SMA50.set_index(pd.DatetimeIndex(df_SMA20_SMA50['Date'].values))
df_SMA20_SMA50['Date'] = df_SMA20_SMA50['Date'].dt.strftime('%Y-%m-%d')

# Lọc 30 ngày

In [15]:
#Set index là Date và lọc 30 ngày 
today_30days = date.today() + datetime.timedelta(days=-30)

df_CCI_30days=df_CCI.sort_index().loc[today_30days:today]
df_CCI_BUY_30days=df_CCI_30days.loc[df_CCI_30days.Advice=='BUY']
list_CCI_duplicate_30days=list(set(df_CCI_BUY_30days['Symbol']))

df_RSI_30days=df_RSI.sort_index().loc[today_30days:today]
df_RSI_BUY_30days=df_RSI_30days.loc[df_RSI_30days.Advice=='BUY']
list_RSI_duplicate_30days=list(set(df_RSI_BUY_30days['Symbol']))

df_MACD_30days=df_MACD.sort_index().loc[today_30days:today]
df_MACD_BUY_30days=df_MACD_30days.loc[df_MACD_30days.Advice=='BUY']
list_MACD_duplicate_30days=list(set(df_MACD_BUY_30days['Symbol']))

df_EMA12_EMA36_30days=df_EMA12_EMA36.sort_index().loc[today_30days:today]
df_EMA12_EMA36_BUY_30days=df_EMA12_EMA36_30days.loc[df_EMA12_EMA36_30days.Advice=='BUY']
list_EMA12_EMA36_duplicate_30days=list(set(df_EMA12_EMA36_BUY_30days['Symbol']))

df_Save_MFI_30days=df_Save_MFI.sort_index().loc[today_30days:today]
df_Save_MFI_BUY_30days=df_Save_MFI_30days.loc[df_Save_MFI_30days.Advice=='BUY']
list_Save_MFI_duplicate_30days=list(set(df_Save_MFI_BUY_30days['Symbol']))

df_BB_30days=df_BB.sort_index().loc[today_30days:today]
df_BB_BUY_30days=df_BB_30days.loc[df_BB_30days.Advice=='BUY']
list_BB_duplicate_30days=list(set(df_BB_BUY_30days['Symbol']))

df_Stochastic_30days=df_Stochastic.sort_index().loc[today_30days:today]
df_Stochastic_BUY_30days=df_Stochastic_30days.loc[df_Stochastic_30days.Advice=='BUY']
list_Stochastic_duplicate_30days=list(set(df_Stochastic_BUY_30days['Symbol']))

df_SAR_30days=df_SAR.sort_index().loc[today_30days:today]
df_SAR_BUY_30days=df_SAR_30days.loc[df_SAR_30days.Advice=='BUY']
list_SAR_duplicate_30days=list(set(df_SAR_BUY_30days['Symbol']))

df_ADX_30days=df_ADX.sort_index().loc[today_30days:today]
df_ADX_BUY_30days=df_ADX_30days.loc[df_ADX_30days.Advice=='Trend']
list_ADX_duplicate_30days=list(set(df_ADX_BUY_30days['Symbol']))

df_SMA20_SMA50_30days=df_SMA20_SMA50.sort_index().loc[today_30days:today]
df_SMA20_SMA50_BUY_30days=df_SMA20_SMA50_30days.loc[df_SMA20_SMA50_30days.Advice=='BUY']
list_SMA20_SMA50_duplicate_30days=list(set(df_SMA20_SMA50_BUY_30days['Symbol']))

today_30days

datetime.date(2021, 9, 18)

# Lọc 3 ngày

In [16]:
#Set index là Date và lọc 3 ngày
today_3days = date.today() + datetime.timedelta(days=-2)

df_CCI_3days=df_CCI.sort_index().loc[today_3days:today]
df_CCI_BUY_3days=df_CCI_3days.loc[df_CCI_3days.Advice=='BUY']
list_CCI_duplicate_3days=list(set(df_CCI_BUY_3days['Symbol']))

df_RSI_3days=df_RSI.sort_index().loc[today_3days:today]
df_RSI_BUY_3days=df_RSI_3days.loc[df_RSI_3days.Advice=='BUY']
list_RSI_duplicate_3days=list(set(df_RSI_BUY_3days['Symbol']))

df_MACD_3days=df_MACD.sort_index().loc[today_3days:today]
df_MACD_BUY_3days=df_MACD_3days.loc[df_MACD_3days.Advice=='BUY']
list_MACD_duplicate_3days=list(set(df_MACD_BUY_3days['Symbol']))

df_EMA12_EMA36_3days=df_EMA12_EMA36.sort_index().loc[today_3days:today]
df_EMA12_EMA36_BUY_3days=df_EMA12_EMA36_3days.loc[df_EMA12_EMA36_3days.Advice=='BUY']
list_EMA12_EMA36_duplicate_3days=list(set(df_EMA12_EMA36_BUY_3days['Symbol']))

df_Save_MFI_3days=df_Save_MFI.sort_index().loc[today_3days:today]
df_Save_MFI_BUY_3days=df_Save_MFI_3days.loc[df_Save_MFI_3days.Advice=='BUY']
list_Save_MFI_duplicate_3days=list(set(df_Save_MFI_BUY_3days['Symbol']))

df_BB_3days=df_BB.sort_index().loc[today_3days:today]
df_BB_BUY_3days=df_BB_3days.loc[df_BB_3days.Advice=='BUY']
list_BB_duplicate_3days=list(set(df_BB_BUY_3days['Symbol']))

df_Stochastic_3days=df_Stochastic.sort_index().loc[today_3days:today]
df_Stochastic_BUY_3days=df_Stochastic_3days.loc[df_Stochastic_3days.Advice=='BUY']
list_Stochastic_duplicate_3days=list(set(df_Stochastic_BUY_3days['Symbol']))

df_SAR_3days=df_SAR.sort_index().loc[today_3days:today]
df_SAR_BUY_3days=df_SAR_3days.loc[df_SAR_3days.Advice=='BUY']
list_SAR_duplicate_3days=list(set(df_SAR_BUY_3days['Symbol']))

df_ADX_3days=df_ADX.sort_index().loc[today_3days:today]
df_ADX_BUY_3days=df_ADX_3days.loc[df_ADX_3days.Advice=='Trend']
list_ADX_duplicate_3days=list(set(df_ADX_BUY_3days['Symbol']))

df_SMA20_SMA50_3days=df_SMA20_SMA50.sort_index().loc[today_3days:today]
df_SMA20_SMA50_BUY_3days=df_SMA20_SMA50_3days.loc[df_SMA20_SMA50_3days.Advice=='BUY']
list_SMA20_SMA50_duplicate_3days=list(set(df_SMA20_SMA50_BUY_3days['Symbol']))

today_3days

datetime.date(2021, 10, 16)

# Lưu file Excel

In [24]:
#lưu file excel đuôi .xls
wb = Workbook()
sheet1 = wb.add_sheet(f'Năm {today.year}')
for i in range(60):
    sheet1.col(i).width = 4700

sheet2 = wb.add_sheet(f'BUY 30 Ngày {today_30days.strftime("%d-%m")}->{today.strftime("%d-%m")}')
for i in range(13):
    sheet2.col(i).width = 4700

sheet3 = wb.add_sheet(f'BUY 3 Ngày {today_3days.day}->{today.day}')
for i in range(13):
    sheet3.col(i).width = 4700

style0 = xlwt.easyxf('font: bold on; pattern: pattern solid, fore_colour gray25')
style1 = xlwt.easyxf('font: bold on')
style2 = xlwt.easyxf('font: bold on, color-index blue')
style3 = xlwt.easyxf('font: color-index blue')
style4 = xlwt.easyxf('font: color-index light_orange')
style5 = xlwt.easyxf('font: bold 1,height 240, name Times New Roman, color-index lavender;')
style6 = xlwt.easyxf('font: color-index green')
style7 = xlwt.easyxf('font: color-index red')
style8 = xlwt.easyxf('font: color-index green, bold on')
style9 = xlwt.easyxf('font: color-index red, bold on')
style10 = xlwt.easyxf('font: color-index gold')
style11 = xlwt.easyxf('font: color-index rose')

style12 = xlwt.easyxf('font: bold on, color-index blue; pattern: pattern solid, fore_colour gray25')
style13 = xlwt.easyxf('font: bold on; pattern: pattern solid, fore_colour pink')

#cố định dòng và cột đầu tiên
sheet1.set_panes_frozen(True)
sheet1.set_horz_split_pos(1) 
sheet2.set_panes_frozen(True)
sheet2.set_horz_split_pos(1) 
sheet3.set_panes_frozen(True)
sheet3.set_horz_split_pos(1) 

################################################## Sheet 1: Thống kê cả năm ############################################################
#Data txt1 Sheet 1
sheet1.write(0, 0, 'Symbol', style0)
sheet1.write(0, 1, 'Indicator', style0)
sheet1.write(0, 2, 'Date', style0)
sheet1.write(0, 3, 'Advice', style0)
txt1_sheet1 = df_CCI.to_numpy()
# Write df_CCI
row = 1
col = 0
for item1,item2,item3,item4 in (txt1_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt2 Sheet 1 
sheet1.write(0, 5, 'Symbol', style0)
sheet1.write(0, 6, 'Indicator', style0)
sheet1.write(0, 7, 'Date', style0)
sheet1.write(0, 8, 'Advice', style0)
txt2_sheet1 = df_RSI.to_numpy()
# Write df_RSI
row = 1
col = 5
for item1,item2,item3,item4 in (txt2_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################
# Data txt3 Sheet 1 
sheet1.write(0, 10, 'Symbol', style0)
sheet1.write(0, 11, 'Indicator', style0)
sheet1.write(0, 12, 'Date', style0)
sheet1.write(0, 13, 'Advice', style0)
txt3_sheet1 = df_MACD.to_numpy()
# Write df_MACD
row = 1
col = 10
for item1,item2,item3,item4 in (txt3_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt4 Sheet 1 
sheet1.write(0, 15, 'Symbol', style0)
sheet1.write(0, 16, 'Indicator', style0)
sheet1.write(0, 17, 'Date', style0)
sheet1.write(0, 18, 'Advice', style0)
txt4_sheet1 = df_EMA12_EMA36.to_numpy()
# Write df_EMA12_EMA36
row = 1
col = 15
for item1,item2,item3,item4 in (txt4_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt5 Sheet 1 
sheet1.write(0, 20, 'Symbol', style0)
sheet1.write(0, 21, 'Indicator', style0)
sheet1.write(0, 22, 'Date', style0)
sheet1.write(0, 23, 'Advice', style0)
txt5_sheet1 = df_Save_MFI.to_numpy()
# Write df_Save_MFI
row = 1
col = 20
for item1,item2,item3,item4 in (txt5_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt6 Sheet 1 
sheet1.write(0, 25, 'Symbol', style0)
sheet1.write(0, 26, 'Indicator', style0)
sheet1.write(0, 27, 'Date', style0)
sheet1.write(0, 28, 'Advice', style0)
txt6_sheet1 = df_BB.to_numpy()
# Write df_BB
row = 1
col = 25
for item1,item2,item3,item4 in (txt6_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt7 Sheet 1 
sheet1.write(0, 30, 'Symbol', style0)
sheet1.write(0, 31, 'Indicator', style0)
sheet1.write(0, 32, 'Date', style0)
sheet1.write(0, 33, 'Advice', style0)
txt7_sheet1 = df_Stochastic.to_numpy()
# Write df_Stochastic
row = 1
col = 30
for item1,item2,item3,item4 in (txt7_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt8 Sheet 1 
sheet1.write(0, 35, 'Symbol', style0)
sheet1.write(0, 36, 'Indicator', style0)
sheet1.write(0, 37, 'Date', style0)
sheet1.write(0, 38, 'Advice', style0)
txt8_sheet1 = df_SAR.to_numpy()
# Write df_SAR
row = 1
col = 35
for item1,item2,item3,item4 in (txt8_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt9 Sheet 1 
sheet1.write(0, 40, 'Symbol', style0)
sheet1.write(0, 41, 'Indicator', style0)
sheet1.write(0, 42, 'Date', style0)
sheet1.write(0, 43, 'Advice', style0)
txt9_sheet1 = df_ADX.to_numpy()
# Write df_ADX
row = 1
col = 40
for item1,item2,item3,item4 in (txt9_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1
#######################################################    
# Data txt10 Sheet 1 
sheet1.write(0, 45, 'Symbol', style0)
sheet1.write(0, 46, 'Indicator', style0)
sheet1.write(0, 47, 'Date', style0)
sheet1.write(0, 48, 'Advice', style0)
txt10_sheet1 = df_SMA20_SMA50.to_numpy()
# Write df_SMA20_SMA50
row = 1
col = 45
for item1,item2,item3,item4 in (txt10_sheet1):
    sheet1.write(row, col, item1)
    sheet1.write(row, col +1, item2)
    sheet1.write(row, col +2, item3)
    sheet1.write(row, col +3, item4)
    row += 1  
    
################################################### Sheet 2: 30days ###########################################################
# Data Sheet 2
sheet2.write(0, 0, 'Chỉ báo CCI', style1)
sheet2.write(0, 1, 'Chỉ báo RSI', style1)
sheet2.write(0, 2, 'Chỉ báo MACD', style1)
sheet2.write(0, 3, 'Chỉ báo EMA12-36', style1)
sheet2.write(0, 4, 'Chỉ báo MFI', style1)
sheet2.write(0, 5, 'Chỉ báo BB', style1)
sheet2.write(0, 6, 'Chỉ báo Stochastic', style1)
sheet2.write(0, 7, 'Chỉ báo SAR', style1)
sheet2.write(0, 8, 'Chỉ báo ADX', style1)
sheet2.write(0, 9, 'Chỉ báo SMA20-50', style1)


col1_sheet2 = list_CCI_duplicate_30days
col2_sheet2 = list_RSI_duplicate_30days
col3_sheet2 = list_MACD_duplicate_30days
col4_sheet2 = list_EMA12_EMA36_duplicate_30days
col5_sheet2 = list_Save_MFI_duplicate_30days
col6_sheet2 = list_BB_duplicate_30days
col7_sheet2 = list_Stochastic_duplicate_30days
col8_sheet2 = list_SAR_duplicate_30days
col9_sheet2 = list_ADX_duplicate_30days
col10_sheet2 = list_SMA20_SMA50_duplicate_30days

# Write Sheet 2
row = 1
col = 0
for item in (col1_sheet2):
    sheet2.write(row, col, item)
    row += 1
row = 1
col = 1
for item in (col2_sheet2):
    sheet2.write(row, col, item)
    row += 1
row = 1
col = 2
for item in (col3_sheet2):
    sheet2.write(row, col, item)
    row += 1  
row = 1
col = 3
for item in (col4_sheet2):
    sheet2.write(row, col, item)
    row += 1

row = 1
col = 4
for item in (col5_sheet2):
    sheet2.write(row, col, item)
    row += 1
    
row = 1
col = 5
for item in (col6_sheet2):
    sheet2.write(row, col, item)
    row += 1
    
row = 1
col = 6
for item in (col7_sheet2):
    sheet2.write(row, col, item)
    row += 1
    
row = 1
col = 7
for item in (col8_sheet2):
    sheet2.write(row, col, item)
    row += 1
    
row = 1
col = 8
for item in (col9_sheet2):
    sheet2.write(row, col, item)
    row += 1   

row = 1
col = 9
for item in (col10_sheet2):
    sheet2.write(row, col, item)
    row += 1   
    

################################################### Sheet 3: 3days ###########################################################
# Data Sheet 3
sheet3.write(0, 0, 'Chỉ báo CCI', style1)
sheet3.write(0, 1, 'Chỉ báo RSI', style1)
sheet3.write(0, 2, 'Chỉ báo MACD', style1)
sheet3.write(0, 3, 'Chỉ báo EMA12-36', style1)
sheet3.write(0, 4, 'Chỉ báo MFI', style1)
sheet3.write(0, 5, 'Chỉ báo BB', style1)
sheet3.write(0, 6, 'Chỉ báo Stochastic', style1)
sheet3.write(0, 7, 'Chỉ báo SAR', style1)
sheet3.write(0, 8, 'Chỉ báo ADX', style1)
sheet3.write(0, 9, 'Chỉ báo SMA20-50', style1)

col1_sheet3 = list_CCI_duplicate_3days
col2_sheet3 = list_RSI_duplicate_3days
col3_sheet3 = list_MACD_duplicate_3days
col4_sheet3 = list_EMA12_EMA36_duplicate_3days
col5_sheet3 = list_Save_MFI_duplicate_3days
col6_sheet3 = list_BB_duplicate_3days
col7_sheet3 = list_Stochastic_duplicate_3days
col8_sheet3 = list_SAR_duplicate_3days
col9_sheet3 = list_ADX_duplicate_3days
col10_sheet3 = list_SMA20_SMA50_duplicate_3days
# Write Sheet 3
row = 1
col = 0
for item in (col1_sheet3):
    sheet3.write(row, col, item)
    row += 1
row = 1
col = 1
for item in (col2_sheet3):
    sheet3.write(row, col, item)
    row += 1
row = 1
col = 2
for item in (col3_sheet3):
    sheet3.write(row, col, item)
    row += 1  
row = 1
col = 3
for item in (col4_sheet3):
    sheet3.write(row, col, item)
    row += 1
    
   
row = 1
col = 4
for item in (col5_sheet3):
    sheet3.write(row, col, item)
    row += 1
    
row = 1
col = 5
for item in (col6_sheet3):
    sheet3.write(row, col, item)
    row += 1

row = 1
col = 6
for item in (col7_sheet3):
    sheet3.write(row, col, item)
    row += 1

row = 1
col = 7
for item in (col8_sheet3):
    sheet3.write(row, col, item)
    row += 1

row = 1
col = 8
for item in (col9_sheet3):
    sheet3.write(row, col, item)
    row += 1
    
row = 1
col = 9
for item in (col10_sheet3):
    sheet3.write(row, col, item)
    row += 1




wb.save(f'E:/OneDrive/CNTT/Project Python/Python Finance/Stock Statistic - Ban dung thu/{str_today_weekday}.xls')
#wb.save(f'G:/Other computers/My Computer/Stock Statistic - Ban dung thu/{str_today_weekday}.xls')

print('Hoàn thành')

Hoàn thành
